<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC2_FD001_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
%pip install scikeras;
%pip install -U tensorflow-addons;

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import datetime

H:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
H:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
H:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.base import BaseEstimator,RegressorMixin

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from tensorflow.keras.metrics import RootMeanSquaredError as RMSE

from keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay

import tensorflow_addons as tfa
from tensorflow_addons.metrics import RSquare as R2



from scikeras.wrappers import KerasRegressor

In [5]:
sns.set_palette('colorblind')

In [6]:
# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [7]:
tf.config.experimental.enable_op_determinism()

# Data Preparation

In [53]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    IN_COLAB = True
    folder="/content/CMaps/"

    # Dataset Download 
    os.system('git clone https://github.com/arthursl12/dataset_2')
    os.system('mv /content/dataset_2/CMaps /content/CMaps')
    os.system('mv /content/dataset_2/data_processing /content/data_processing')
    os.system('rm -rf dataset_2')
else:
    print('Not running on CoLab')
    IN_COLAB = False
    folder="CMaps/"
    %cd dataset_2/

Not running on CoLab
[WinError 2] The system cannot find the file specified: 'dataset_2/'
C:\Users\Arthur Lima\POC\dataset_2


In [54]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [55]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [56]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(1,folder=folder)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640


## Preprocessing

### Validation Set

In [57]:
from sklearn.model_selection import GroupShuffleSplit

def train_val_split(train):
    gss = GroupShuffleSplit(n_splits=1, train_size=0.80, random_state=42)  
    for idx_train, idx_val in gss.split(train,groups=train["unit_number"]):
        # print('train_split_engines', train.iloc[idx_train]['unit_number'].unique(), '\n')
        # print('validate_split_engines', train.iloc[idx_val]['unit_number'].unique(), '\n')

        df_train = train.iloc[idx_train].copy()
        df_val = train.iloc[idx_val].copy()

    return df_train, df_val

In [58]:
df_train, df_val = train_val_split(train)

In [59]:
X_t_train, y_t_train = proc.X_y_train_divide(df_train)
X_t_val, y_t_val = proc.X_y_train_divide(df_val)

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [60]:
test.shape, y_test.shape

((13096, 26), (100, 1))

In [61]:
test_last = proc.transform_test(test)
test_last.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.58,1581.22,1398.91,14.62,21.61,554.42,2388.08,9056.40,1.3,...,521.79,2388.06,8130.11,8.4024,0.03,393,2388,100.0,38.81,23.3552
1,518.67,642.55,1586.59,1410.83,14.62,21.61,553.52,2388.10,9044.77,1.3,...,521.74,2388.09,8126.90,8.4505,0.03,391,2388,100.0,38.81,23.2618
2,518.67,642.88,1589.75,1418.89,14.62,21.61,552.59,2388.16,9049.26,1.3,...,520.83,2388.14,8131.46,8.4119,0.03,395,2388,100.0,38.93,23.2740
3,518.67,642.78,1594.53,1406.88,14.62,21.61,552.64,2388.13,9051.30,1.3,...,521.88,2388.11,8133.64,8.4634,0.03,395,2388,100.0,38.58,23.2581
4,518.67,642.27,1589.94,1419.36,14.62,21.61,553.29,2388.10,9053.99,1.3,...,521.00,2388.15,8125.74,8.4362,0.03,394,2388,100.0,38.75,23.4117


In [62]:
X_test = test_last

### Remaining Useful Life (RUL)

In [63]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,191
1,1,2,190
2,1,3,189
3,1,4,188
4,1,5,187


## Attributes and target separation

In [64]:
X_train, y_train = proc.X_y_train_divide(train)

In [65]:
y_train.head()

,RUL
0,191
1,190
2,189
3,188
4,187


In [66]:
X_train.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


## Training and Evaluation functions

In [67]:
eval = Evaluation()

In [68]:
search = HyperparameterSearch()

# MLP Construction

## Callbacks

In [24]:
# Early Stopping Callback
es = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                      patience=5, restore_best_weights=True)

In [25]:
# Printing Callback
def printLog(epoch, logs):
    print(
        f"E {epoch+1}\t: loss={logs['loss']:.3f}, "+
        f"rmse={logs['root_mean_squared_error']:.3f}, "+
        f"r2={logs['r_square']:.3f}; "+
        f"v_loss={logs['val_loss']:.3f}, "+
        f"v_rmse={logs['val_root_mean_squared_error']:.3f}, "+
        f"v_r2={logs['val_r_square']:.3f}; "
    )

printerCallback = LambdaCallback(on_epoch_end=printLog)

## Auxiliary HyperParameters

In [26]:
layer_sizes=[16,32,64,128,256,512]

In [27]:
# Exponential Decay Schedules
ED1 = ExponentialDecay(initial_learning_rate=1e-2, name="ED1",
                       decay_steps=100000, decay_rate=0.96)
ED2 = ExponentialDecay(initial_learning_rate=1e-2, name="ED2",
                       decay_steps=100000, decay_rate=0.8)
ED3 = ExponentialDecay(initial_learning_rate=1e-1, name="ED3",
                       decay_steps=100000, decay_rate=0.96)

In [28]:
# Generation of list combinations
from itertools import chain, permutations

def all_permutations(lst, size):
    result = list(chain.from_iterable([permutations(lst, x) for x in range(len(lst)+1)]))
    out = []
    for r in result:
        if (len(r) == size):
            out.append(list(r))
    return out

## Constants

In [29]:
# X_train must include indices
train3 = train.copy()
X_train_ = train3.drop(columns=[index_cols[1]]+["RUL"])

In [30]:
INPUT_SHAPE = 0

## Wrapper

In [50]:
from sys import is_finalizing
from sklearn.metrics import r2_score

class MLPWrapperRegressor(BaseEstimator,RegressorMixin):
    def __init__(self, basemodel=None, clip_y=-1,
                 include_settings=False, poly_degree=1,
                 scaler=StandardScaler()):
        # Base parameters
        self.basemodel = basemodel
        self.clip_y = clip_y
        self.poly_degree = poly_degree
        self.include_settings = include_settings

        # Column indexers
        self.feature_cols = sensors_cols
        if(include_settings):
            self.feature_cols = settings_cols + self.feature_cols
        self.base_feature_cols = self.feature_cols

        # Scaler and PolyFeatures transformers
        self.scaler = scaler
        self.polyft = PolynomialFeatures(degree=self.poly_degree, 
                                         include_bias=False)

    def clean_cols(self,df):
        if("unit_number" in df.columns): del df["unit_number"]
        if("unit_number" in df.columns): del df["unit_number"]
        if("time" in df.columns): del df["time"]
        if((not self.include_settings)): 
            for col in settings_cols:
                if(col in df.columns): del df[col]
        return df

    def fit(self, X=None, y=None):
        # Drop irrelevant column
        data = pd.DataFrame(X).copy()
        data = self.clean_cols(data)
        
        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.fit_transform(data)
        data = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out())
        self.feature_cols = list(self.polyft.get_feature_names_out())

        # Scale the data
        data = self.scaler.fit_transform(data)
        data = pd.DataFrame(data, 
                            columns=self.scaler.get_feature_names_out())
        X_train = data.copy()
        
        # Clip and transform labels
        data2 = pd.DataFrame(y, columns=["RUL"]).copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_train = data2

        # Update input shape for future use
        global INPUT_SHAPE
        INPUT_SHAPE = X_train.shape[1]

        # Fit model
        self.basemodel.fit(X_train,y_train)
        return self
        
    def predict(self, X=None):
        # Perform transformation, if not done
        if (len(X.columns) != len(self.feature_cols)):
            X_train = self.transform_features(X)
        else:
            X_train = X
        return self.basemodel.predict(X_train)

    def transform_features(self, df):
        # Drop irrelevant column
        data = pd.DataFrame(df).copy()
        data = self.clean_cols(data)
        
        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.transform(data)
        data = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out())
        # self.feature_cols = list(self.polyft.get_feature_names_out())

        # Scale the data
        data = self.scaler.transform(data)
        data = pd.DataFrame(data, 
                            columns=self.scaler.get_feature_names_out())
        return data

    def score(self, X, y, sample_weight=None):
        X_test = self.transform_features(X)

        # Clip and transform labels
        data2 = pd.DataFrame(y, columns=["RUL"]).copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_test = data2

        # Predict on test data
        y_pred = self.predict(X_test)
        return r2_score(y_test, y_pred, sample_weight=sample_weight)

## Test Data

In [32]:
def scale_test(test,model):
    test2 = pd.DataFrame(test)
    test2 = model.clean_cols(test2)

    # Apply polynomial features
    transf = model.polyft.transform(test2)
    test2 = pd.DataFrame(transf, 
                         columns=model.polyft.get_feature_names_out())

    # Scale the data (with train data parameters)
    test2 = model.scaler.transform(test2)
    test2 = pd.DataFrame(test2, 
                         columns=model.polyft.get_feature_names_out())
    return test2

## Constructor

In [33]:
def create_model(optim=Adam, learning_rate=1e-3, 
                 layer1=32  , activation1="tanh"    , dropout1=0.1,
                 layer2=None, activation2="tanh"    , dropout2=0.1,
                 layer3=None, activation3="tanh"    , dropout3=0.1,
                 layer4=None, activation4="tanh"    , dropout4=0.1,
                 print_summary=False, loss='mean_squared_error',
                 metrics=[tf.keras.metrics.MeanSquaredError()]):
    model = Sequential()

    # Fully Connected Layer
    model.add(Dense(layer1, input_dim=INPUT_SHAPE, activation=activation1))
    model.add(Dropout(dropout1))

    # Additional hidden layers
    if(layer2 is not None):
        model.add(Dense(layer2, activation=activation2))
        model.add(Dropout(dropout2))
        if (layer3 is not None):
            model.add(Dense(layer3, activation=activation3))
            model.add(Dropout(dropout3))
            if (layer4 is not None):
                model.add(Dense(layer4, activation=activation4))
                model.add(Dropout(dropout4))

    # Output Layer
    model.add(Dense(1))

    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate), 
                  metrics=metrics)
    
    if(print_summary): model.summary()
    return model

# MLP-1

## Linear RUL

Score: 0.6275937811619965  
Test: 0.504
```
('basemodel__batch_size', 262),
('basemodel__epochs', 47),
('basemodel__model__activation', 'relu'),
('basemodel__model__dropout', 0.42198962995190514),
('basemodel__model__layer1', 512),
('basemodel__model__learning_rate', 0.00528953958947019),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.17975244796806303),
('scaler', StandardScaler())
```

In [69]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=262,
                           epochs=47,
                           model__activation1='relu',
                           model__dropout1=0.42198962995190514, 
                           model__layer1=512, 
                           model__learning_rate=0.00528953958947019,
                           model__optim=Adam,
                           validation_split=0.17975244796806303, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=262, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=47, model=<function create_model at 0x000001C61804DDC0>, model__activation1='relu', model__dropout1=0.42198962995190514, model__layer1=512, model__learning_rate=0.00528953958947019, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C6532841C0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C618000A90>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.17975244796806303, verbose=0))

In [70]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 512)               11264     
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 1)                 513       
                                                                 
Total params: 11,777
Trainable params: 11,777
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=7768.790, rmse=88.141, r2=-0.742; v_loss=4409.638, v_rmse=66.405, v_r2=0.258; 
E 2	: loss=2299.978, rmse=47.958, r2=0.484; v_loss=2736.304, v_rmse=52.310, v_r2=0.539; 
E 3	: loss=1662.483, rmse=40.774, r2=0.627; v_loss=2625.097, v_rmse=51.236, v_r2=0.558; 
E 4	: loss=1608.881, rmse=40.111, r2=0.639; v_

In [71]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.506,RMSE=-29.199
Finished: 2022-10-16 08:54:12.730080


## Non-Linear RUL

Score: 0.8670568151811345  
Test: 0.865
```
('basemodel__batch_size', 32),
('basemodel__epochs', 50),
('basemodel__model__activation', 'sigmoid'),
('basemodel__model__dropout', 0.1),
('basemodel__model__layer1', 450),
('basemodel__model__learning_rate', 0.006977518145392822),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('clip_y', 80),
('scaler', MinMaxScaler())
```

In [72]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=50,
                           model__activation1='sigmoid',
                           model__dropout1=0.1, 
                           model__layer1=450, 
                           model__learning_rate=0.006977518145392822,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=50, model=<function create_model at 0x000001C61804DDC0>, model__activation1='sigmoid', model__dropout1=0.1, model__layer1=450, model__learning_rate=0.006977518145392822, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C617DF6AF0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C6194942B0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=80, scaler=MinMaxScaler())

In [73]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 450)               9900      
                                                                 
 dropout_13 (Dropout)        (None, 450)               0         
                                                                 
 dense_21 (Dense)            (None, 1)                 451       
                                                                 
Total params: 10,351
Trainable params: 10,351
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=343.262, rmse=18.527, r2=0.428; v_loss=121.074, v_rmse=11.003, v_r2=0.783; 
E 2	: loss=111.671, rmse=10.567, r2=0.814; v_loss=98.067, v_rmse=9.903, v_r2=0.824; 
E 3	: loss=95.680, rmse=9.782, r2=0.841; v_loss=94.655, v_rmse=9.729, v_r2=0.831; 
E 4	: loss=92.671, rmse=9.627, r2=0.846; v_loss=95.667, v_r

In [75]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.855,RMSE=-10.152
Finished: 2022-10-16 08:57:04.260956


## PolyFeatures + Linear RUL 


Score: 0.6307902416968273  
Test: 0.289
```
('basemodel__batch_size', 197),
('basemodel__epochs', 41),
('basemodel__model__activation', 'relu'),
('basemodel__model__dropout', 0.1),
('basemodel__model__layer1', 400),
('basemodel__model__learning_rate', 0.01),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('poly_degree', 2),
('scaler', StandardScaler())
```

In [76]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=197,
                           epochs=41,
                           model__activation1='relu',
                           model__dropout1=0.1, 
                           model__layer1=400, 
                           model__learning_rate=0.01,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=197, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=41, model=<function create_model at 0x000001C61804DDC0>, model__activation1='relu', model__dropout1=0.1, model__layer1=400, model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C6186A1AC0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C618643B80>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    poly_degree=2)

In [77]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 400)               101200    
                                                                 
 dropout_14 (Dropout)        (None, 400)               0         
                                                                 
 dense_23 (Dense)            (None, 1)                 401       
                                                                 
Total params: 101,601
Trainable params: 101,601
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=3590.689, rmse=59.922, r2=0.211; v_loss=2748.295, v_rmse=52.424, v_r2=0.561; 
E 2	: loss=1942.870, rmse=44.078, r2=0.573; v_loss=2533.745, v_rmse=50.336, v_r2=0.596; 
E 3	: loss=1824.145, rmse=42.710, r2=0.599; v_loss=2450.530, v_rmse=49.503, v_r2=0.609; 
E 4	: loss=1733.372, rmse=41.634, r2=0.619; v

In [78]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.597,RMSE=-26.379
Finished: 2022-10-16 08:57:37.851678


## PolyFeatures + Non-Linear RUL


Score: 0.8693404879366771  
Test: 0.874
```
('basemodel__batch_size', 278),
('basemodel__epochs', 50),
('basemodel__model__activation', 'relu'),
('basemodel__model__dropout', 0.1),
('basemodel__model__layer1', 469),
('basemodel__model__learning_rate', 0.002874922426530888),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('clip_y', 80),
('poly_degree', 2),
('scaler', StandardScaler())
```

In [80]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=278,
                           epochs=50,
                           model__activation1='relu',
                           model__dropout1=0.1, 
                           model__layer1=469, 
                           model__learning_rate=0.002874922426530888,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=278, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=50, model=<function create_model at 0x000001C61804DDC0>, model__activation1='relu', model__dropout1=0.1, model__layer1=469, model__learning_rate=0.002874922426530888, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C618879DC0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C6188938B0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=80, poly_degree=2)

In [81]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 469)               118657    
                                                                 
 dropout_15 (Dropout)        (None, 469)               0         
                                                                 
 dense_25 (Dense)            (None, 1)                 470       
                                                                 
Total params: 119,127
Trainable params: 119,127
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=1629.454, rmse=40.366, r2=-1.716; v_loss=780.958, v_rmse=27.946, v_r2=-0.398; 
E 2	: loss=491.348, rmse=22.166, r2=0.181; v_loss=314.425, v_rmse=17.732, v_r2=0.437; 
E 3	: loss=282.066, rmse=16.795, r2=0.530; v_loss=232.004, v_rmse=15.232, v_r2=0.585; 
E 4	: loss=220.318, rmse=14.843, r2=0.633; v_los

In [82]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.875,RMSE=-9.428
Finished: 2022-10-16 08:58:17.045025


# MLP-2

## Linear RUL

Score: 0.6272918246191982  
Test: 0.
```
('basemodel__batch_size', 180),
             ('basemodel__epochs', 10),
             ('basemodel__model__activation1', 'selu'),
             ('basemodel__model__activation2', 'sigmoid'),
             ('basemodel__model__dropout1', 0.6877162153707456),
             ('basemodel__model__dropout2', 0.23474535539749616),
             ('basemodel__model__layer1', 468),
             ('basemodel__model__layer2', 435),
             ('basemodel__model__learning_rate', 0.009683115627232612),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.1),
             ('scaler', StandardScaler())
```

In [83]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=180,
                           epochs=10,
                           model__activation1='selu',
                           model__activation2='sigmoid',
                           model__dropout1=0.6877162153707456, 
                           model__dropout2=0.23474535539749616, 
                           model__layer1=468, 
                           model__layer2=435, 
                           model__learning_rate=0.009683115627232612,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=180, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=10, model=<function create_model at 0x000001C61804DDC0>, model__activation1='selu', model__activation2='sigmoid', model__dropout1=0.6877162153707456, model__dropo..., model__layer1=468, model__layer2=435, model__learning_rate=0.009683115627232612, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C6189F2640>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C618895100>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0))

In [84]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 468)               10296     
                                                                 
 dropout_16 (Dropout)        (None, 468)               0         
                                                                 
 dense_27 (Dense)            (None, 435)               204015    
                                                                 
 dropout_17 (Dropout)        (None, 435)               0         
                                                                 
 dense_28 (Dense)            (None, 1)                 436       
                                                                 
Total params: 214,747
Trainable params: 214,747
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=4170.941, rmse=64.583, r2=0.083; v_l

In [85]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.466,RMSE=-30.355
Finished: 2022-10-16 08:58:43.443147


## Non-Linear RUL

Score: 0.8708908017407038  
Test: 0.
```
('basemodel__batch_size', 272),
             ('basemodel__epochs', 39),
             ('basemodel__model__activation1', 'elu'),
             ('basemodel__model__activation2', 'sigmoid'),
             ('basemodel__model__dropout1', 0.3474531287065963),
             ('basemodel__model__dropout2', 0.36614542849836884),
             ('basemodel__model__layer1', 512),
             ('basemodel__model__layer2', 139),
             ('basemodel__model__learning_rate', 0.0015318728990539984),
             ('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
             ('basemodel__validation_split', 0.23383223141315412),
             ('clip_y', 80),
             ('scaler', StandardScaler())
```

In [87]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=272,
                           epochs=39,
                           model__activation1='elu',
                           model__activation2='sigmoid',
                           model__dropout1=0.3474531287065963, 
                           model__dropout2=0.36614542849836884, 
                           model__layer1=512, 
                           model__layer2=139, 
                           model__learning_rate=0.0015318728990539984,
                           model__optim=RMSprop,
                           validation_split=0.23383223141315412, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=272, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=39, model=<function create_model at 0x000001C61804DDC0>, model__activation1='elu', model__activation2='sigmoid', model__dropout1=0.3474531287065963, model__dropou...139, model__learning_rate=0.0015318728990539984, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C653CF98B0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C653CF9EE0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.23383223141315412, verbose=0),
                    clip_y=80)

In [88]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 512)               11264     
                                                                 
 dropout_18 (Dropout)        (None, 512)               0         
                                                                 
 dense_30 (Dense)            (None, 139)               71307     
                                                                 
 dropout_19 (Dropout)        (None, 139)               0         
                                                                 
 dense_31 (Dense)            (None, 1)                 140       
                                                                 
Total params: 82,711
Trainable params: 82,711
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=3658.698, rmse=60.487, r2=-5.073; v_lo

In [89]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.873,RMSE=-9.515
Finished: 2022-10-16 08:59:26.065457


## PolyFeatures + Linear RUL

Score: 0.6262323565109925  
Test: 0.
```
('basemodel__batch_size', 32),
             ('basemodel__epochs', 30),
             ('basemodel__model__activation1', 'tanh'),
             ('basemodel__model__activation2', 'elu'),
             ('basemodel__model__dropout1', 0.9),
             ('basemodel__model__dropout2', 0.1),
             ('basemodel__model__layer1', 320),
             ('basemodel__model__layer2', 512),
             ('basemodel__model__learning_rate', 0.0027118397103578972),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.1),
             ('poly_degree', 3),
             ('scaler', StandardScaler())
```


In [90]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=30,
                           model__activation1='tanh',
                           model__activation2='elu',
                           model__dropout1=0.9, 
                           model__dropout2=0.1, 
                           model__layer1=320, 
                           model__layer2=512, 
                           model__learning_rate=0.0027118397103578972,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=30, model=<function create_model at 0x000001C61804DDC0>, model__activation1='tanh', model__activation2='elu', model__dropout1=0.9, model__dropout2=0.1, model__layer1=320, model__layer2=512, model__learning_rate=0.0027118397103578972, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C6542B0670>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C65413C970>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    poly_degree=3)

In [91]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 320)               647680    
                                                                 
 dropout_20 (Dropout)        (None, 320)               0         
                                                                 
 dense_33 (Dense)            (None, 512)               164352    
                                                                 
 dropout_21 (Dropout)        (None, 512)               0         
                                                                 
 dense_34 (Dense)            (None, 1)                 513       
                                                                 
Total params: 812,545
Trainable params: 812,545
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=3144.958, rmse=56.080, r2=0.309; v_l

In [92]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.210,RMSE=-36.946
Finished: 2022-10-16 09:00:58.155336


## PolyFeatures + Non-Linear RUL


Score: 0.8653619825760903  
Test: 0.
```
('basemodel__batch_size', 235),
             ('basemodel__epochs', 50),
             ('basemodel__model__activation1', 'sigmoid'),
             ('basemodel__model__activation2', 'elu'),
             ('basemodel__model__dropout1', 0.1),
             ('basemodel__model__dropout2', 0.22990220831017208),
             ('basemodel__model__layer1', 81),
             ('basemodel__model__layer2', 211),
             ('basemodel__model__learning_rate', 0.01),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.1),
             ('clip_y', 80),
             ('poly_degree', 3),
             ('scaler', MinMaxScaler())
```

In [93]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=235,
                           epochs=50,
                           model__activation1='sigmoid',
                           model__activation2='elu',
                           model__dropout1=0.1, 
                           model__dropout2=0.22990220831017208, 
                           model__layer1=81, 
                           model__layer2=211, 
                           model__learning_rate=0.01,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=235, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=50, model=<function create_model at 0x000001C61804DDC0>, model__activation1='sigmoid', model__activation2='elu', model__dropout1=0.1, model__dropout2=0.2299022083..._layer1=81, model__layer2=211, model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C7808D0670>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C7808D0AC0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=80, poly_degree=3)

In [94]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 81)                163944    
                                                                 
 dropout_22 (Dropout)        (None, 81)                0         
                                                                 
 dense_36 (Dense)            (None, 211)               17302     
                                                                 
 dropout_23 (Dropout)        (None, 211)               0         
                                                                 
 dense_37 (Dense)            (None, 1)                 212       
                                                                 
Total params: 181,458
Trainable params: 181,458
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=499.261, rmse=22.344, r2=0.168; v_lo

In [95]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.800,RMSE=-11.927
Finished: 2022-10-16 09:01:16.164693


# MLP-3

## Linear RUL

Score: 0.6120201145167702    
Test: 0.
```
('basemodel__batch_size', 244),
             ('basemodel__epochs', 25),
             ('basemodel__model__activation1', 'elu'),
             ('basemodel__model__activation2', 'tanh'),
             ('basemodel__model__activation3', 'elu'),
             ('basemodel__model__dropout1', 0.1),
             ('basemodel__model__dropout2', 0.6452357951609182),
             ('basemodel__model__dropout3', 0.10365428493216054),
             ('basemodel__model__layer1', 33),
             ('basemodel__model__layer2', 216),
             ('basemodel__model__layer3', 480),
             ('basemodel__model__learning_rate', 0.0023087723482705077),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.17402892098258826),
             ('scaler', MinMaxScaler())
```

In [97]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=244,
                           epochs=25,
                           model__activation1='elu',
                           model__activation2='tanh',
                           model__activation3='elu',
                           model__dropout1=0.1, 
                           model__dropout2=0.6452357951609182, 
                           model__dropout3=0.1, 
                           model__layer1=33, 
                           model__layer2=216, 
                           model__layer3=480, 
                           model__learning_rate=0.0023087723482705077,
                           model__optim=Adam,
                           validation_split=0.17402892098258826, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=244, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=25, model=<function create_model at 0x000001C61804DDC0>, model__activation1='elu', model__activation2='tanh', model__activation3='elu', model__dropout1=0.1, model_...del__learning_rate=0.0023087723482705077, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C78DD48BE0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C78DD63F10>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.17402892098258826, verbose=0),
                    scaler=MinMaxScaler())

In [98]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 33)                726       
                                                                 
 dropout_24 (Dropout)        (None, 33)                0         
                                                                 
 dense_39 (Dense)            (None, 216)               7344      
                                                                 
 dropout_25 (Dropout)        (None, 216)               0         
                                                                 
 dense_40 (Dense)            (None, 480)               104160    
                                                                 
 dropout_26 (Dropout)        (None, 480)               0         
                                                                 
 dense_41 (Dense)            (None, 1)               

In [99]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.442,RMSE=-31.051
Finished: 2022-10-16 09:02:30.136596


## Non-Linear RUL

Score: 0.858429482908221    
Test: 0.
```
('basemodel__batch_size', 387),
             ('basemodel__epochs', 37),
             ('basemodel__model__activation1', 'tanh'),
             ('basemodel__model__activation2', 'selu'),
             ('basemodel__model__activation3', 'elu'),
             ('basemodel__model__dropout1', 0.6025012184092352),
             ('basemodel__model__dropout2', 0.584225016100644),
             ('basemodel__model__dropout3', 0.6585832537238048),
             ('basemodel__model__layer1', 462),
             ('basemodel__model__layer2', 287),
             ('basemodel__model__layer3', 303),
             ('basemodel__model__learning_rate', 0.0047806961601746184),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.5866937471711482),
             ('clip_y', 80),
             ('scaler', StandardScaler())
```

In [100]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=387,
                           epochs=37,
                           model__activation1='tanh',
                           model__activation2='selu',
                           model__activation3='elu',
                           model__dropout1=0.6025012184092352, 
                           model__dropout2=0.584225016100644, 
                           model__dropout3=0.6585832537238048, 
                           model__layer1=462, 
                           model__layer2=287, 
                           model__layer3=383,  
                           model__learning_rate=0.0047806961601746184,
                           model__optim=Adam,
                           validation_split=0.5866937471711482, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=387, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=37, model=<function create_model at 0x000001C61804DDC0>, model__activation1='tanh', model__activation2='selu', model__activation3='elu', model__dropout1=0.6025012...layer3=383, model__learning_rate=0.0047806961601746184, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C79D032F10>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C79D032430>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.5866937471711482, verbose=0),
                    clip_y=80)

In [101]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 462)               10164     
                                                                 
 dropout_27 (Dropout)        (None, 462)               0         
                                                                 
 dense_43 (Dense)            (None, 287)               132881    
                                                                 
 dropout_28 (Dropout)        (None, 287)               0         
                                                                 
 dense_44 (Dense)            (None, 383)               110304    
                                                                 
 dropout_29 (Dropout)        (None, 383)               0         
                                                                 
 dense_45 (Dense)            (None, 1)               

In [102]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.870,RMSE=-9.612
Finished: 2022-10-16 09:15:43.674605


## PolyFeatures + Linear RUL

Score: 0.6222047496761983    
Test: 0.
```
('basemodel__batch_size', 315),
             ('basemodel__epochs', 27),
             ('basemodel__model__activation1', 'elu'),
             ('basemodel__model__activation2', 'selu'),
             ('basemodel__model__activation3', 'selu'),
             ('basemodel__model__dropout1', 0.3672917021354055),
             ('basemodel__model__dropout2', 0.2585591117469549),
             ('basemodel__model__dropout3', 0.1),
             ('basemodel__model__layer1', 456),
             ('basemodel__model__layer2', 174),
             ('basemodel__model__layer3', 479),
             ('basemodel__model__learning_rate', 0.0007321508636106998),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.1),
             ('poly_degree', 2),
             ('scaler', StandardScaler())
```

In [103]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=315,
                           epochs=27,
                           model__activation1='elu',
                           model__activation2='selu',
                           model__activation3='selu',
                           model__dropout1=0.3672917021354055, 
                           model__dropout2=0.2585591117469549, 
                           model__dropout3=0.1, 
                           model__layer1=456, 
                           model__layer2=174, 
                           model__layer3=479, 
                           model__learning_rate=0.0007321508636106998,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=315, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=27, model=<function create_model at 0x000001C61804DDC0>, model__activation1='elu', model__activation2='selu', model__activation3='selu', model__dropout1=0.3672917...174, model__layer3=479, model__learning_rate=0.0007321508636106998, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C6187E5DF0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C6187E5EE0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    poly_degree=2)

In [104]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 456)               115368    
                                                                 
 dropout_30 (Dropout)        (None, 456)               0         
                                                                 
 dense_47 (Dense)            (None, 174)               79518     
                                                                 
 dropout_31 (Dropout)        (None, 174)               0         
                                                                 
 dense_48 (Dense)            (None, 479)               83825     
                                                                 
 dropout_32 (Dropout)        (None, 479)               0         
                                                                 
 dense_49 (Dense)            (None, 1)               

In [105]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.416,RMSE=-31.766
Finished: 2022-10-16 09:17:15.837217


## PolyFeatures + Non-Linear RUL


Score: 0.8679661672429307   
Test: 0.
```
('basemodel__batch_size', 283),
             ('basemodel__epochs', 14),
             ('basemodel__model__activation1', 'tanh'),
             ('basemodel__model__activation2', 'relu'),
             ('basemodel__model__activation3', 'elu'),
             ('basemodel__model__dropout1', 0.1),
             ('basemodel__model__dropout2', 0.53425143831743),
             ('basemodel__model__dropout3', 0.1),
             ('basemodel__model__layer1', 512),
             ('basemodel__model__layer2', 512),
             ('basemodel__model__layer3', 305),
             ('basemodel__model__learning_rate', 0.009573430971057319),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.1),
             ('clip_y', 80),
             ('poly_degree', 3),
             ('scaler', StandardScaler())
```

In [106]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=283,
                           epochs=14,
                           model__activation1='tanh',
                           model__activation2='relu',
                           model__activation3='elu',
                           model__dropout1=0.1, 
                           model__dropout2=0.53425143831743, 
                           model__dropout3=0.1, 
                           model__layer1=512, 
                           model__layer2=512, 
                           model__layer3=305, 
                           model__learning_rate=0.009573430971057319,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=283, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=14, model=<function create_model at 0x000001C61804DDC0>, model__activation1='tanh', model__activation2='relu', model__activation3='elu', model__dropout1=0.1, model...__layer3=305, model__learning_rate=0.009573430971057319, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C653FB9BE0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C653FB9E80>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=80, poly_degree=3)

In [107]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 512)               1036288   
                                                                 
 dropout_33 (Dropout)        (None, 512)               0         
                                                                 
 dense_51 (Dense)            (None, 512)               262656    
                                                                 
 dropout_34 (Dropout)        (None, 512)               0         
                                                                 
 dense_52 (Dense)            (None, 305)               156465    
                                                                 
 dropout_35 (Dropout)        (None, 305)               0         
                                                                 
 dense_53 (Dense)            (None, 1)               

In [108]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.758,RMSE=-13.132
Finished: 2022-10-16 09:19:08.802380


# MLP-4

## Linear RUL


Score: 0.8684757521280048    
Test: 0.
```
('basemodel__batch_size', 127),
             ('basemodel__epochs', 17),
             ('basemodel__model__activation1', 'relu'),
             ('basemodel__model__activation2', 'sigmoid'),
             ('basemodel__model__activation3', 'elu'),
             ('basemodel__model__activation4', 'elu'),
             ('basemodel__model__dropout1', 0.405658593077186),
             ('basemodel__model__dropout2', 0.3589949636608206),
             ('basemodel__model__dropout3', 0.4633066337493309),
             ('basemodel__model__dropout4', 0.32587881379978234),
             ('basemodel__model__layer1', 308),
             ('basemodel__model__layer2', 442),
             ('basemodel__model__layer3', 301),
             ('basemodel__model__layer4', 153),
             ('basemodel__model__learning_rate', 0.0038661981642895585),
             ('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
             ('basemodel__validation_split', 0.14630628708453258),
             ('scaler', StandardScaler())
```

In [109]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=127,
                           epochs=17,
                           model__activation1='relu',
                           model__activation2='sigmoid',
                           model__activation3='elu',
                           model__activation4='elu',
                           model__dropout1=0.405658593077186, 
                           model__dropout2=0.3589949636608206, 
                           model__dropout3=0.4633066337493309, 
                           model__dropout4=0.32587881379978234, 
                           model__layer1=308, 
                           model__layer2=442, 
                           model__layer3=301, 
                           model__layer4=153, 
                           model__learning_rate=0.0038661981642895585,
                           model__optim=RMSprop,
                           validation_split=0.14630628708453258, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=127, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=17, model=<function create_model at 0x000001C61804DDC0>, model__activation1='relu', model__activation2='sigmoid', model__activation3='elu', model__activation4='el...l__layer4=153, model__learning_rate=0.0038661981642895585, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C79FF38E80>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C79FF38190>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.14630628708453258, verbose=0))

In [110]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 308)               6776      
                                                                 
 dropout_36 (Dropout)        (None, 308)               0         
                                                                 
 dense_55 (Dense)            (None, 442)               136578    
                                                                 
 dropout_37 (Dropout)        (None, 442)               0         
                                                                 
 dense_56 (Dense)            (None, 301)               133343    
                                                                 
 dropout_38 (Dropout)        (None, 301)               0         
                                                                 
 dense_57 (Dense)            (None, 153)             

In [111]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.610,RMSE=-25.966
Finished: 2022-10-16 09:22:02.970379


## Non-Linear RUL

Score: 0.8527803619368491    
Test: 0.
```
('basemodel__batch_size', 344),
             ('basemodel__epochs', 27),
             ('basemodel__model__activation1', 'elu'),
             ('basemodel__model__activation2', 'selu'),
             ('basemodel__model__activation3', 'sigmoid'),
             ('basemodel__model__activation4', 'sigmoid'),
             ('basemodel__model__dropout1', 0.6749954668968566),
             ('basemodel__model__dropout2', 0.2102017209355273),
             ('basemodel__model__dropout3', 0.1),
             ('basemodel__model__dropout4', 0.47711054364433003),
             ('basemodel__model__layer1', 464),
             ('basemodel__model__layer2', 371),
             ('basemodel__model__layer3', 16),
             ('basemodel__model__layer4', 476),
             ('basemodel__model__learning_rate', 0.004154798051053557),
             ('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
             ('basemodel__validation_split', 0.1),
             ('clip_y', 80),
             ('scaler', StandardScaler())
```


In [112]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=344,
                           epochs=27,
                           model__activation1='elu',
                           model__activation2='selu',
                           model__activation3='sigmoid',
                           model__activation4='sigmoid',
                           model__dropout1=0.6749954668968566, 
                           model__dropout2=0.2102017209355273, 
                           model__dropout3=0.1, 
                           model__dropout4=0.47711054364433003, 
                           model__layer1=464, 
                           model__layer2=371, 
                           model__layer3=16, 
                           model__layer4=476, 
                           model__learning_rate=0.004154798051053557,
                           model__optim=RMSprop,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=344, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=27, model=<function create_model at 0x000001C61804DDC0>, model__activation1='elu', model__activation2='selu', model__activation3='sigmoid', model__activation4='si...16, model__layer4=476, model__learning_rate=0.004154798051053557, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C7A06093D0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C7A06200A0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=80)

In [113]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_59 (Dense)            (None, 464)               10208     
                                                                 
 dropout_40 (Dropout)        (None, 464)               0         
                                                                 
 dense_60 (Dense)            (None, 371)               172515    
                                                                 
 dropout_41 (Dropout)        (None, 371)               0         
                                                                 
 dense_61 (Dense)            (None, 16)                5952      
                                                                 
 dropout_42 (Dropout)        (None, 16)                0         
                                                                 
 dense_62 (Dense)            (None, 476)             

In [114]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.856,RMSE=-10.118
Finished: 2022-10-16 09:23:59.379892


## PolyFeatures + Linear RUL

Score: 0.6121049470412723    
Test: 0.
```
('basemodel__batch_size', 249),
             ('basemodel__epochs', 37),
             ('basemodel__model__activation1', 'relu'),
             ('basemodel__model__activation2', 'relu'),
             ('basemodel__model__activation3', 'elu'),
             ('basemodel__model__activation4', 'sigmoid'),
             ('basemodel__model__dropout1', 0.62787764969972),
             ('basemodel__model__dropout2', 0.9),
             ('basemodel__model__dropout3', 0.6788232857905743),
             ('basemodel__model__dropout4', 0.1),
             ('basemodel__model__layer1', 512),
             ('basemodel__model__layer2', 512),
             ('basemodel__model__layer3', 92),
             ('basemodel__model__layer4', 133),
             ('basemodel__model__learning_rate', 0.0022168133279249643),
             ('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
             ('basemodel__validation_split', 0.1),
             ('poly_degree', 2),
             ('scaler', StandardScaler())
```


In [115]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=249,
                           epochs=37,
                           model__activation1='relu',
                           model__activation2='relu',
                           model__activation3='elu',
                           model__activation4='sigmoid',
                           model__dropout1=0.62787764969972, 
                           model__dropout2=0.9, 
                           model__dropout3=0.6788232857905743, 
                           model__dropout4=0.1, 
                           model__layer1=512, 
                           model__layer2=512, 
                           model__layer3=92, 
                           model__layer4=133, 
                           model__learning_rate=0.0022168133279249643,
                           model__optim=RMSprop,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=249, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=37, model=<function create_model at 0x000001C61804DDC0>, model__activation1='relu', model__activation2='relu', model__activation3='elu', model__activation4='sigmo...del__layer4=133, model__learning_rate=0.0022168133279249643, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C7A096E3A0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C7A096E640>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.1, verbose=0),
                    poly_degree=2)

In [116]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_64 (Dense)            (None, 512)               129536    
                                                                 
 dropout_44 (Dropout)        (None, 512)               0         
                                                                 
 dense_65 (Dense)            (None, 512)               262656    
                                                                 
 dropout_45 (Dropout)        (None, 512)               0         
                                                                 
 dense_66 (Dense)            (None, 92)                47196     
                                                                 
 dropout_46 (Dropout)        (None, 92)                0         
                                                                 
 dense_67 (Dense)            (None, 133)             

In [117]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.177,RMSE=-37.689
Finished: 2022-10-16 09:25:47.158321


## PolyFeatures + Non-Linear RUL


Score: 0.8425016957390049  
Test: 0.
```
('basemodel__batch_size', 32),
             ('basemodel__epochs', 50),
             ('basemodel__model__activation1', 'elu'),
             ('basemodel__model__activation2', 'tanh'),
             ('basemodel__model__activation3', 'tanh'),
             ('basemodel__model__activation4', 'tanh'),
             ('basemodel__model__dropout1', 0.9),
             ('basemodel__model__dropout2', 0.1),
             ('basemodel__model__dropout3', 0.9),
             ('basemodel__model__dropout4', 0.1),
             ('basemodel__model__layer1', 512),
             ('basemodel__model__layer2', 512),
             ('basemodel__model__layer3', 512),
             ('basemodel__model__layer4', 512),
             ('basemodel__model__learning_rate', 0.0001),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.1),
             ('clip_y', 80),
             ('poly_degree', 2),
             ('scaler', StandardScaler())
```

In [118]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=50,
                           model__activation1='elu',
                           model__activation2='tanh',
                           model__activation3='tanh',
                           model__activation4='tanh',
                           model__dropout1=0.9, 
                           model__dropout2=0.1, 
                           model__dropout3=0.9, 
                           model__dropout4=0.1, 
                           model__layer1=512, 
                           model__layer2=512, 
                           model__layer3=512, 
                           model__layer4=512, 
                           model__learning_rate=0.0001,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x000001C617C32790>, <keras.callbacks.LambdaCallback object at 0x000001C618017520>], epochs=50, model=<function create_model at 0x000001C61804DDC0>, model__activation1='elu', model__activation2='tanh', model__activation3='tanh', model__activation4='tanh',...yer3=512, model__layer4=512, model__learning_rate=0.0001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001C7A090CE80>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001C7A31C0760>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=80, poly_degree=2)

In [119]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_69 (Dense)            (None, 512)               129536    
                                                                 
 dropout_48 (Dropout)        (None, 512)               0         
                                                                 
 dense_70 (Dense)            (None, 512)               262656    
                                                                 
 dropout_49 (Dropout)        (None, 512)               0         
                                                                 
 dense_71 (Dense)            (None, 512)               262656    
                                                                 
 dropout_50 (Dropout)        (None, 512)               0         
                                                                 
 dense_72 (Dense)            (None, 512)             

In [120]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.843,RMSE=-10.585
Finished: 2022-10-16 09:30:52.014184


# OFF

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.5,
''mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'mlp__model__learning_rate': 0.01, 
'mlp__model__layer_nodes': [128],
'mlp__model__dropout': 0.2,
'mlp__model__activation': 'relu',
'mlp__epochs': 20, 
'mlp__batch_size': 64}
```

In [ ]:
model = Pipeline([
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                model__print_summary=True,
                                validation_split=0.5, 
                                model__optim=RMSprop,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[128],
                                model__dropout=0.2,
                                model__activation="relu",
                                epochs=20, 
                                batch_size=64))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=64, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f406e3db050>, <keras.callbacks.LambdaCallback object at 0x7f406008ff10>], epochs=20, model=<function create_model at 0x7f3fd95efc20>, model__activation='relu', model__dropout=0.2, model__layer_nodes=[128], model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f3fd9df3950>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f3fbcfbe450>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, validation_split=0.5, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

E 0	: loss=2735.080, rmse=52.298, r2=0.294; v_loss=2654.072, v_rmse=51.518, v_r2=0.522; 
E 1	: loss=1063.129, rmse=32.606, r2=0.725; v_loss=2618.092, v_rmse=51.167, v_r2=0.528; 
E 2	: loss=1061.893, rmse=32.587, r2=0.726; v_loss=2480.985, v_rmse=49.809, v_r2=0.553; 
E 3	: loss=1049.969, rmse=32.403, r2=0.729; v_loss=2760.811, v_rmse=52.543, v_r2=0.502; 
E 4	: loss=1051.995, rmse=32.434, r2=0.728; v_loss=2802.044, v_rmse=52.934, v_r2=0.495; 
E 5	: loss=1048.582, rmse=32.382, r2=0.729; v_loss=2526.332, v_rmse=50.263, v_r2=0.545; 
E 6	: loss=1045.081, rmse=32.328, r2=0.730; v_loss=2638.387, v_rmse=51.365, v_r2=0.524; 
E 7	: loss=1035.534, rmse=32.180, r2=0.733; v_loss=2611.114, v_rmse=51.099, v_r2=0.529; 
E 8	: loss=1032.359, rmse=32.130, r2=0.733; v_loss=2670.928, v_rmse=51.681, v_r2=0.519; 
E 9	: loss=1038.198, rmse=32.221, r2=0.732; v_loss=2574.217, v_rmse=50.737, v_r2=0.536; 
E 10	: loss=1037.728, rmse=32.214, r2=0.732; v_loss=2707.143, v_rmse=52.030, v_r2=0.512; 
E 11	: loss=1036.660

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.488,RMSE=-29.721
Finished: 2022-09-15 12:35:08.268111


## Non-linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model,verbose=0, callbacks=[es], 
                                model__degree=1,
                                validation_split=0.2, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f3fd4507690>], model=<function create_model at 0x7f3fc11ce170>, model__degree=1, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f3fbc3bafd0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f3fbc023410>], validation_split=0.2, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f40e9f9bc20>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~15min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": all_permutations(layer_sizes, 1),
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 128}, 'trf_reg__regressor__validation_split': 0.2, 'trf_reg__regressor__model__optim': <class 'kera

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 128}, 
'trf_reg__regressor__validation_split': 0.2, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'trf_reg__regressor__model__learning_rate': 0.01, 
'trf_reg__regressor__model__layer_nodes': [512],
'trf_reg__regressor__model__dropout': 0.5,
'trf_reg__regressor__model__activation': 'tanh',
'trf_reg__regressor__epochs': 30,
'trf_reg__regressor__batch_size': 64,
'scaler': MinMaxScaler()}
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,  
                                validation_split=0.2, 
                                model__optim=RMSprop,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[512],
                                model__dropout=0.5,
                                model__activation="tanh",
                                epochs=30, 
                                batch_size=64),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':128})))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=64, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f3fd4507690>, <keras.callbacks.LambdaCallback object at 0x7f3fbc023810>], epochs=30, model=<function create_model at 0x7f3fc11ce170>, model__activation='tanh', model__dro...cs=[<keras.metrics.RootMeanSquaredError object at 0x7f3fbeb848d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f3fbc2c8e10>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.2, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f40e9f9bc20>,
                                                                            kw_args={'a_max': 128,
                                                                             

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_767"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1534 (Dense)          (None, 512)               11264     
                                                                 
 dropout_767 (Dropout)       (None, 512)               0         
                                                                 
 dense_1535 (Dense)          (None, 1)                 513       
                                                                 
Total params: 11,777
Trainable params: 11,777
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=838.904, rmse=28.964, r2=0.541; v_loss=468.957, v_rmse=21.655, v_r2=0.742; 
E 1	: loss=493.887, rmse=22.224, r2=0.730; v_loss=440.370, v_rmse=20.985, v_r2=0.758; 
E 2	: loss=458.042, rmse=21.402, r2=0.749; v_loss=465.912, v_rmse=21.585, v_r2=0.744; 
E 3	: loss=437.639, rmse=20.920, r2=0.760; v_loss=4

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.808,RMSE=-17.733
Finished: 2022-09-15 13:31:13.681717


The best result we had so far in training and in testing

## PolyFeatures(2) + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=2,
                                verbose=0, callbacks=[es],  
                                ))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>], model=<function create_model at 0x7f1959d24050>, model__degree=2, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18c7fc1610>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18c7f49810>], verbose=0))])

In [ ]:
# ~12min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__degree": [2],
        "mlp__model__optim":[Adam,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": all_permutations(layer_sizes, 1),
        "mlp__model__activation": ["relu","tanh", "sigmoid"]    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits


----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 'mlp__model__learning_rate': 0.01, 'mlp__model__layer_nodes': [512], 'mlp__model__dropout': 0.4, 'mlp__model__degree': 2, 'mlp__model__activation': 'sigmoid', 'mlp__epochs': 50, 'mlp__batch_size': 512}
Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('sca

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'mlp__model__learning_rate': 0.01, 
'mlp__model__layer_nodes': [512], 
'mlp__model__dropout': 0.4, 
'mlp__model__degree': 2, 
'mlp__model__activation': 'sigmoid', 
'mlp__epochs': 50, 
'mlp__batch_size': 512}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,   
                                validation_split=0.2, 
                                model__optim=RMSprop,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[512],
                                model__dropout=0.4,
                                model__degree=2,
                                model__activation="sigmoid",
                                epochs=50, 
                                batch_size=512))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=512, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>, <keras.callbacks.LambdaCallback object at 0x7f195016cf10>], epochs=50, model=<function create_model at 0x7f1959d24050>, model__activation='sigmoid', model__degree=2, model__dropout=0.4, model__layer_nodes=[512], model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18c42d1a50>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18c42d1c50>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.2, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_253"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_506 (Dense)           (None, 512)               129536    
                                                                 
 dropout_253 (Dropout)       (None, 512)               0         
                                                                 
 dense_507 (Dense)           (None, 1)                 513       
                                                                 
Total params: 130,049
Trainable params: 130,049
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=5057.620, rmse=71.117, r2=-0.163; v_loss=3196.043, v_rmse=56.534, v_r2=0.484; 
E 1	: loss=1720.632, rmse=41.481, r2=0.604; v_loss=2960.801, v_rmse=54.413, v_r2=0.522; 
E 2	: loss=1653.111, rmse=40.658, r2=0.620; v_loss=2736.813, v_rmse=52.315, v_r2=0.558; 
E 3	: loss=1638.268, rmse=40.476, r2=0.623

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.333,RMSE=-33.941
Finished: 2022-09-16 11:24:17.967695


Bad generalization using 2 degree polynomial

## PolyFeatures(2) + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=2,
                                verbose=0, callbacks=[es],  
                                ),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>], model=<function create_model at 0x7f1959d24050>, model__degree=2, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18b1e9b7d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18b1e9b590>], verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f19d590bc20>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~10min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__model__degree": [2],
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": all_permutations(layer_sizes, 1),
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 'trf_reg__regressor__validation_split': 0.5, 'trf_reg__regressor__model__optim': <class 'kera

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 
'trf_reg__regressor__validation_split': 0.5, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'trf_reg__regressor__model__learning_rate': 0.01, 
'trf_reg__regressor__model__layer_nodes': [16], 
'trf_reg__regressor__model__dropout': 0.2, 
'trf_reg__regressor__model__degree': 2, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 50, 
'trf_reg__regressor__batch_size': 32, 
'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2,include_bias=False)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback], 
                                model__print_summary=True,   
                                validation_split=0.5, 
                                model__optim=Adam,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[16],
                                model__dropout=0.2,
                                model__degree=2,
                                model__activation="relu",
                                epochs=50, 
                                batch_size=32),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':135})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>, <keras.callbacks.LambdaCallback object at 0x7f195016cf10>], epochs=50, model=<function create_model..._metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18b1fc0c10>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18a9c57d10>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__print_summary=True, validation_split=0.5, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f19d590bc20>,
                                                                            kw_args={'a_max': 135,
                                                               

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_507"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1014 (Dense)          (None, 16)                4048      
                                                                 
 dropout_507 (Dropout)       (None, 16)                0         
                                                                 
 dense_1015 (Dense)          (None, 1)                 17        
                                                                 
Total params: 4,065
Trainable params: 4,065
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=2450.159, rmse=49.499, r2=-0.206; v_loss=1878.002, v_rmse=43.336, v_r2=0.081; 
E 1	: loss=1160.745, rmse=34.070, r2=0.428; v_loss=835.719, v_rmse=28.909, v_r2=0.591; 
E 2	: loss=836.896, rmse=28.929, r2=0.588; v_loss=676.510, v_rmse=26.010, v_r2=0.669; 
E 3	: loss=722.874, rmse=26.886, r2=0.644; v_loss

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.775,RMSE=-19.536
Finished: 2022-09-16 11:50:29.954140


Good generalization, but worse than regular features

## PolyFeatures(3) + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=3,
                                verbose=0, callbacks=[es],  
                                ))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>], model=<function create_model at 0x7f1959d24050>, model__degree=3, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18b190e390>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18c89e9910>], verbose=0))])

In [ ]:
# ~15min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__degree": [3],
        "mlp__model__optim":[Adam,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": all_permutations(layer_sizes, 1),
        "mlp__model__activation": ["relu","tanh", "sigmoid"]    
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 'mlp__model__learning_rate': 

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'mlp__model__learning_rate': 0.01, 
'mlp__model__layer_nodes': [512], 
'mlp__model__dropout': 0.4, 
'mlp__model__degree': 3, 
'mlp__model__activation': 'sigmoid', 
'mlp__epochs': 50, 
'mlp__batch_size': 512}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,   
                                validation_split=0.2, 
                                model__optim=RMSprop,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[512],
                                model__dropout=0.4,
                                model__degree=3,
                                model__activation="sigmoid",
                                epochs=50, 
                                batch_size=512))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=512, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>, <keras.callbacks.LambdaCallback object at 0x7f195016cf10>], epochs=50, model=<function create_model at 0x7f1959d24050>, model__activation='sigmoid', model...3, model__dropout=0.4, model__layer_nodes=[512], model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18c57d9e90>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18b02418d0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.2, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_761"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1522 (Dense)          (None, 512)               1036288   
                                                                 
 dropout_761 (Dropout)       (None, 512)               0         
                                                                 
 dense_1523 (Dense)          (None, 1)                 513       
                                                                 
Total params: 1,036,801
Trainable params: 1,036,801
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=5458.847, rmse=73.884, r2=-0.256; v_loss=3615.433, v_rmse=60.128, v_r2=0.416; 
E 1	: loss=1796.632, rmse=42.387, r2=0.587; v_loss=2949.010, v_rmse=54.305, v_r2=0.523; 
E 2	: loss=1698.000, rmse=41.207, r2=0.609; v_loss=2761.931, v_rmse=52.554, v_r2=0.554; 
E 3	: loss=1675.702, rmse=40.935, r2=0

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.509,RMSE=-29.129
Finished: 2022-09-16 12:08:01.694012


Similar architecture to the NL 1-degree polynomial, but worse results. Slight improvement over 1-degree Linear, better than 2-degree.

## PolyFeatures(3) + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=3,
                                verbose=0, callbacks=[es],  
                                ),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>], model=<function create_model at 0x7f1959d24050>, model__degree=3, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18b0121350>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18b0121410>], verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f19d590bc20>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~20min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__model__degree": [3],
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": all_permutations(layer_sizes, 1),
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 'trf_reg__regressor__validation_split': 0.5, 'trf_reg__regressor__model__optim': <class 'kera

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 
'trf_reg__regressor__validation_split': 0.5, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'trf_reg__regressor__model__learning_rate': 0.01, 
'trf_reg__regressor__model__layer_nodes': [16], 
'trf_reg__regressor__model__dropout': 0.2, 
'trf_reg__regressor__model__degree': 3, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 50, 
'trf_reg__regressor__batch_size': 32, 
'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3,include_bias=False)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback], 
                                model__print_summary=True,   
                                validation_split=0.5, 
                                model__optim=Adam,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[16],
                                model__dropout=0.2,
                                model__degree=3,
                                model__activation="relu",
                                epochs=50, 
                                batch_size=32),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':135})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>, <keras.callbacks.LambdaCallback object at 0x7f195016cf10>], epochs=50, model=<function cre..._metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18b1862b50>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18b0092c50>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__print_summary=True, validation_split=0.5, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f19d590bc20>,
                                                                            kw_args={'a_max': 135,
                                                              

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_1015"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2030 (Dense)          (None, 16)                32384     
                                                                 
 dropout_1015 (Dropout)      (None, 16)                0         
                                                                 
 dense_2031 (Dense)          (None, 1)                 17        
                                                                 
Total params: 32,401
Trainable params: 32,401
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=1975.571, rmse=44.447, r2=0.027; v_loss=1402.159, v_rmse=37.445, v_r2=0.314; 
E 1	: loss=1257.777, rmse=35.465, r2=0.381; v_loss=1110.082, v_rmse=33.318, v_r2=0.457; 
E 2	: loss=1061.762, rmse=32.585, r2=0.477; v_loss=951.694, v_rmse=30.850, v_r2=0.534; 
E 3	: loss=926.742, rmse=30.442, r2=0.544; v_

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.782,RMSE=-19.248
Finished: 2022-09-16 12:36:55.847390


Didn't improve by using more features, actually worsened a bit

# MLP-2

## Linear RUL

In [ ]:
model = Pipeline([
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=1,
                                verbose=0, callbacks=[es],  
                                ))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>], model=<function create_model at 0x7f1959d24050>, model__degree=1, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18b66c0e90>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18b0476990>], verbose=0))])

In [ ]:
# ~15min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__optim":[Adam,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": all_permutations(layer_sizes, 2),
        "mlp__model__activation": ["relu","tanh", "sigmoid"]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.5, 'mlp__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 'mlp__model__learning_rate': 0.01, 

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.5, 
'mlp__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'mlp__model__learning_rate': 0.01, 
'mlp__model__layer_nodes': [512, 32], 
'mlp__model__dropout': 0.1, 
'mlp__model__activation': 'relu', 
'mlp__epochs': 50, 
'mlp__batch_size': 64}
```

In [ ]:
model = Pipeline([
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,  
                                validation_split=0.5, 
                                model__optim=Adam,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[512,32],
                                model__dropout=0.1,
                                model__activation="relu",
                                epochs=50, 
                                batch_size=64))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=64, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>, <keras.callbacks.LambdaCallback object at 0x7f195016cf10>], epochs=50, model=<function create_model at 0x7f1959d24050>, model__activation='relu', model__dropout=0.1, model__layer_nodes=[512, 32], model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18b3319410>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18a727aa90>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__print_summary=True, validation_split=0.5, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_1270"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2794 (Dense)          (None, 512)               11264     
                                                                 
 dropout_1524 (Dropout)      (None, 512)               0         
                                                                 
 dense_2795 (Dense)          (None, 32)                16416     
                                                                 
 dropout_1525 (Dropout)      (None, 32)                0         
                                                                 
 dense_2796 (Dense)          (None, 1)                 33        
                                                                 
Total params: 27,713
Trainable params: 27,713
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=2102.799, rmse=45.856, r2=0.457; v_l

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.528,RMSE=-28.550
Finished: 2022-09-16 12:55:16.728608


Better than one layer, but still bad

## Non-linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model,verbose=0, callbacks=[es], 
                                model__degree=1,
                                validation_split=0.2, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>], model=<function create_model at 0x7f1959d24050>, model__degree=1, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18a727a490>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18a727a390>], validation_split=0.2, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f19d590bc20>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~15min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": all_permutations(layer_sizes, 2),
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 137}, 'trf_reg__regressor__validation_split': 0.4, 'trf_reg__regressor__model__optim': <class 'kera

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 137}, 
'trf_reg__regressor__validation_split': 0.4, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'trf_reg__regressor__model__learning_rate': 0.01, 
'trf_reg__regressor__model__layer_nodes': [512, 256], 
'trf_reg__regressor__model__dropout': 0.4, 
'trf_reg__regressor__model__activation': 'tanh', 
'trf_reg__regressor__epochs': 20, 
'trf_reg__regressor__batch_size': 256, 
'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,  
                                validation_split=0.4, 
                                model__optim=RMSprop,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[512,256],
                                model__dropout=0.4,
                                model__activation="tanh",
                                epochs=20, 
                                batch_size=64),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':137})))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=64, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>, <keras.callbacks.LambdaCallback object at 0x7f195016cf10>], epochs=20, model=<function create_model at 0x7f1959d24050>, model__activation='tanh', model__d...cs=[<keras.metrics.RootMeanSquaredError object at 0x7f18b1baf850>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18b1bafbd0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.4, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f19d590bc20>,
                                                                            kw_args={'a_max': 137,
                                                                             

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_1524"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3556 (Dense)          (None, 512)               11264     
                                                                 
 dropout_2032 (Dropout)      (None, 512)               0         
                                                                 
 dense_3557 (Dense)          (None, 256)               131328    
                                                                 
 dropout_2033 (Dropout)      (None, 256)               0         
                                                                 
 dense_3558 (Dense)          (None, 1)                 257       
                                                                 
Total params: 142,849
Trainable params: 142,849
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=942.921, rmse=30.707, r2=0.550; v_

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.793,RMSE=-18.829
Finished: 2022-09-16 13:19:49.918665


Similar to 1-layer, but slightly worse

## PolyFeatures(2) + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=2,
                                verbose=0, callbacks=[es],  
                                ))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>], model=<function create_model at 0x7fa530f75e60>, model__degree=2, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa530f7a890>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa530f64e90>], verbose=0))])

In [ ]:
# ~20min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__degree": [2],
        "mlp__model__optim":[Adam,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": all_permutations(layer_sizes, 2),
        "mlp__model__activation": ["relu","tanh", "sigmoid"]    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits


----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'scaler': MinMaxScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 'mlp__model__learning_rate': 0.0001, 'mlp__model__layer_nodes': [256, 64], 'mlp__model__dropout': 0.1, 'mlp__model__degree': 2, 'mlp__model__activation': 'relu', 'mlp__epochs': 50, 'mlp__batch_size': 32}
Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('sca

Best Model:
```
{'scaler': MinMaxScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'mlp__model__learning_rate': 0.0001, 
'mlp__model__layer_nodes': [256, 64], 
'mlp__model__dropout': 0.1, 
'mlp__model__degree': 2, 
'mlp__model__activation': 'relu', 
'mlp__epochs': 50, 
'mlp__batch_size': 32}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler'  , MinMaxScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,   
                                validation_split=0.2, 
                                model__optim=RMSprop,
                                model__learning_rate=0.0001, 
                                model__layer_nodes=[256,64],
                                model__dropout=0.1,
                                model__degree=2,
                                model__activation="relu",
                                epochs=50, 
                                batch_size=32))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>, <keras.callbacks.LambdaCallback object at 0x7fa520152cd0>], epochs=50, model=<function create_model at 0x7fa530f75e60>, model__activation='relu', model__degree=2, model__dropout=0.1, model__layer_nodes=[256, 64], model__learning_rate=0.0001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa48c3ec4d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48655c4d0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.2, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_253"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_759 (Dense)           (None, 256)               64768     
                                                                 
 dropout_506 (Dropout)       (None, 256)               0         
                                                                 
 dense_760 (Dense)           (None, 64)                16448     
                                                                 
 dropout_507 (Dropout)       (None, 64)                0         
                                                                 
 dense_761 (Dense)           (None, 1)                 65        
                                                                 
Total params: 81,281
Trainable params: 81,281
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=8284.538, rmse=91.019, r2=-0.906; v_l

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.419,RMSE=-31.683
Finished: 2022-09-16 16:31:59.488230


Bad generalization using 2 degree polynomial and more layers. The overall result is slightly better than one layer only

## PolyFeatures(2) + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=2,
                                verbose=0, callbacks=[es],  
                                ),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>], model=<function create_model at 0x7fa530f75e60>, model__degree=2, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa48b030150>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa4865ebbd0>], verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fa5acad2c20>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~25min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__model__degree": [2],
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": all_permutations(layer_sizes, 2),
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 119}, 'trf_reg__regressor__validation_split': 0.1, 'trf_reg__regressor__model__optim': <class 'kera

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 119}, 
'trf_reg__regressor__validation_split': 0.1, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'trf_reg__regressor__model__learning_rate': 0.001, 
'trf_reg__regressor__model__layer_nodes': [128, 32], 
'trf_reg__regressor__model__dropout': 0.1, 
'trf_reg__regressor__model__degree': 2, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 50, 
'trf_reg__regressor__batch_size': 32, 
'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2,include_bias=False)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback], 
                                model__print_summary=True,   
                                validation_split=0.1, 
                                model__optim=RMSprop,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[128,32],
                                model__dropout=0.1,
                                model__degree=2,
                                model__activation="relu",
                                epochs=50, 
                                batch_size=32),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':119})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>, <keras.callbacks.LambdaCallback object at 0x7fa520152cd0>], epochs=50, model=<function create_model...cs=[<keras.metrics.RootMeanSquaredError object at 0x7fa48cd1e690>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48cd1edd0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.1, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fa5acad2c20>,
                                                                            kw_args={'a_max': 119,
                                                               

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_507"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1521 (Dense)          (None, 128)               32384     
                                                                 
 dropout_1014 (Dropout)      (None, 128)               0         
                                                                 
 dense_1522 (Dense)          (None, 32)                4128      
                                                                 
 dropout_1015 (Dropout)      (None, 32)                0         
                                                                 
 dense_1523 (Dense)          (None, 1)                 33        
                                                                 
Total params: 36,545
Trainable params: 36,545
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=1413.037, rmse=37.590, r2=0.096; v_lo

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.840,RMSE=-15.687
Finished: 2022-09-16 17:00:08.047966


Best result so far

## PolyFeatures(3) + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=3,
                                verbose=0, callbacks=[es],  
                                ))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>], model=<function create_model at 0x7fa530f75e60>, model__degree=3, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa48c38d290>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48c38de50>], verbose=0))])

In [ ]:
# ~25min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__degree": [3],
        "mlp__model__optim":[Adam,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": all_permutations(layer_sizes, 2),
        "mlp__model__activation": ["relu","tanh", "sigmoid"]    
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'scaler': MinMaxScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 'mlp__model__learning_rate': 0.

Best Model:
```
{'scaler': MinMaxScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'mlp__model__learning_rate': 0.0001, 
'mlp__model__layer_nodes': [256, 64], 
'mlp__model__dropout': 0.1, 
'mlp__model__degree': 3, 
'mlp__model__activation': 'relu', 
'mlp__epochs': 50, 
'mlp__batch_size': 32}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,   
                                validation_split=0.2, 
                                model__optim=RMSprop,
                                model__learning_rate=0.0001, 
                                model__layer_nodes=[256,64],
                                model__dropout=0.1,
                                model__degree=3,
                                model__activation="relu",
                                epochs=50, 
                                batch_size=32))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>, <keras.callbacks.LambdaCallback object at 0x7fa520152cd0>], epochs=50, model=<function create_model at 0x7fa530f75e60>, model__activation='relu', model__de...el__dropout=0.1, model__layer_nodes=[256, 64], model__learning_rate=0.0001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa49d378810>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48e3923d0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.2, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_761"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2283 (Dense)          (None, 256)               518144    
                                                                 
 dropout_1522 (Dropout)      (None, 256)               0         
                                                                 
 dense_2284 (Dense)          (None, 64)                16448     
                                                                 
 dropout_1523 (Dropout)      (None, 64)                0         
                                                                 
 dense_2285 (Dense)          (None, 1)                 65        
                                                                 
Total params: 534,657
Trainable params: 534,657
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=4928.556, rmse=70.204, r2=-0.134; v

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.551,RMSE=-27.830
Finished: 2022-09-16 17:31:09.294675


Similar architecture to the Linear 1-layer and Linear 1-poly 1-layer. Better than 2-degree (like in MLP-1) 

## PolyFeatures(3) + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=3,
                                verbose=0, callbacks=[es],  
                                ),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>], model=<function create_model at 0x7fa530f75e60>, model__degree=3, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa48c9bd6d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa49c272090>], verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fa5acad2c20>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~30min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__model__degree": [3],
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": all_permutations(layer_sizes, 2),
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 119}, 'trf_reg__regressor__validation_split': 0.1, 'trf_reg__regressor__model__optim': <class 'kera

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 119}, 
'trf_reg__regressor__validation_split': 0.1, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'trf_reg__regressor__model__learning_rate': 0.001, 
'trf_reg__regressor__model__layer_nodes': [128, 32], 
'trf_reg__regressor__model__dropout': 0.1, 
'trf_reg__regressor__model__degree': 3, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 50, 
'trf_reg__regressor__batch_size': 32, 
'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3,include_bias=False)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback], 
                                model__print_summary=True,   
                                validation_split=0.1, 
                                model__optim=RMSprop,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[128,32],
                                model__dropout=0.1,
                                model__degree=3,
                                model__activation="relu",
                                epochs=50, 
                                batch_size=32),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':119})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>, <keras.callbacks.LambdaCallback object at 0x7fa520152cd0>], epochs=50, model=<function cre...cs=[<keras.metrics.RootMeanSquaredError object at 0x7fa48c754950>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48b232c10>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.1, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fa5acad2c20>,
                                                                            kw_args={'a_max': 119,
                                                              

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_1015"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3045 (Dense)          (None, 128)               259072    
                                                                 
 dropout_2030 (Dropout)      (None, 128)               0         
                                                                 
 dense_3046 (Dense)          (None, 32)                4128      
                                                                 
 dropout_2031 (Dropout)      (None, 32)                0         
                                                                 
 dense_3047 (Dense)          (None, 1)                 33        
                                                                 
Total params: 263,233
Trainable params: 263,233
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=1069.400, rmse=32.702, r2=0.316; v

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.840,RMSE=-15.697
Finished: 2022-09-16 18:05:09.679005


Best result from 1 and 2 layers

# MLP-3

## Linear RUL

In [ ]:
model = Pipeline([
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=1,
                                verbose=0, callbacks=[es],  
                                ))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>], model=<function create_model at 0x7fa530f75e60>, model__degree=1, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa48a8d4410>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48a8d46d0>], verbose=0))])

In [ ]:
# ~15min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__optim":[Adam,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": all_permutations(layer_sizes, 3),
        "mlp__model__activation": ["relu","tanh", "sigmoid"]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 'mlp__model__learning_rate': 0.01, 

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'mlp__model__learning_rate': 0.01, 
'mlp__model__layer_nodes': [64, 128, 256], 
'mlp__model__dropout': 0.4, 
'mlp__model__activation': 'tanh', 
'mlp__epochs': 10,
'mlp__batch_size': 32}
```

In [ ]:
model = Pipeline([
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,  
                                validation_split=0.2, 
                                model__optim=Adam,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[64,128,256],
                                model__dropout=0.4,
                                model__activation="tanh",
                                epochs=10, 
                                batch_size=32))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>, <keras.callbacks.LambdaCallback object at 0x7fa520152cd0>], epochs=10, model=<function create_model at 0x7fa530f75e60>, model__activation='tanh', model__dropout=0.4, model__layer_nodes=[64, 128, 256], model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa49d2d4d10>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48aca6210>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__print_summary=True, validation_split=0.2, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_1775"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6084 (Dense)          (None, 64)                1408      
                                                                 
 dropout_4309 (Dropout)      (None, 64)                0         
                                                                 
 dense_6085 (Dense)          (None, 128)               8320      
                                                                 
 dropout_4310 (Dropout)      (None, 128)               0         
                                                                 
 dense_6086 (Dense)          (None, 256)               33024     
                                                                 
 dropout_4311 (Dropout)      (None, 256)               0         
                                                                 
 dense_6087 (Dense)          (None, 1)             

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.467,RMSE=-30.332
Finished: 2022-09-16 19:26:20.413887


As bad as one layer.

## Non-linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model,verbose=0, callbacks=[es], 
                                model__degree=1,
                                validation_split=0.2, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>], model=<function create_model at 0x7fa530f75e60>, model__degree=1, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa48a8c5390>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48a8c5250>], validation_split=0.2, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fa5acad2c20>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~15min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": all_permutations(layer_sizes, 3),
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 143}, 'trf_reg__regressor__validation_split': 0.5, 'trf_reg__regressor__model__optim': <class 'kera

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 143},
'trf_reg__regressor__validation_split': 0.5, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'trf_reg__regressor__model__learning_rate': 0.01, 
'trf_reg__regressor__model__layer_nodes': [64, 128, 32], 
'trf_reg__regressor__model__dropout': 0.1, 
'trf_reg__regressor__model__activation': 'tanh', 
'trf_reg__regressor__epochs': 30, 
'trf_reg__regressor__batch_size': 64, 
'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,  
                                validation_split=0.5, 
                                model__optim=RMSprop,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[64,128,32],
                                model__dropout=0.1,
                                model__activation="tanh",
                                epochs=30, 
                                batch_size=64),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':143})))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=64, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>, <keras.callbacks.LambdaCallback object at 0x7fa520152cd0>], epochs=30, model=<function create_model at 0x7fa530f75e60>, model__activation='tanh', model__d...cs=[<keras.metrics.RootMeanSquaredError object at 0x7fa4862c6b50>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa488d28d50>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.5, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fa5acad2c20>,
                                                                            kw_args={'a_max': 143,
                                                                             

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_1776"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6088 (Dense)          (None, 64)                1408      
                                                                 
 dropout_4312 (Dropout)      (None, 64)                0         
                                                                 
 dense_6089 (Dense)          (None, 128)               8320      
                                                                 
 dropout_4313 (Dropout)      (None, 128)               0         
                                                                 
 dense_6090 (Dense)          (None, 32)                4128      
                                                                 
 dropout_4314 (Dropout)      (None, 32)                0         
                                                                 
 dense_6091 (Dense)          (None, 1)             

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.762,RMSE=-20.274
Finished: 2022-09-16 19:28:22.263094


Similar to 1-layer, but slightly worse

## PolyFeatures(2) + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=2,
                                verbose=0, callbacks=[es],  
                                ))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>], model=<function create_model at 0x7fa530f75e60>, model__degree=2, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa488c3a650>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa488c3a150>], verbose=0))])

In [ ]:
# ~15min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__degree": [2],
        "mlp__model__optim":[Adam,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": all_permutations(layer_sizes, 3),
        "mlp__model__activation": ["relu","tanh", "sigmoid"]    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.1, 'mlp__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 'mlp__model__learning_rate': 0.01, 

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.1, 
'mlp__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'mlp__model__learning_rate': 0.01,
'mlp__model__layer_nodes': [256, 16, 32], 
'mlp__model__dropout': 0.1, 
'mlp__model__degree': 2, 
'mlp__model__activation': 'tanh', 
'mlp__epochs': 50, 
'mlp__batch_size': 64}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler'  , MinMaxScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,   
                                validation_split=0.1, 
                                model__optim=Adam,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[256,16,32],
                                model__dropout=0.1,
                                model__degree=2,
                                model__activation="tanh",
                                epochs=50, 
                                batch_size=64))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(batch_size=64, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa4863d6610>, <keras.callbacks.LambdaCallback object at 0x7fa488bc7690>], epochs=50, model=<function create_model at 0x7fa4888839e0>, model__activation='tanh', model__degree=2, model__dropout=0.1, model__layer_nodes=[256, 16, 32], model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa48f66d710>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48ae161d0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__print_summary=True, validation_split=0.1, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_1778"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6096 (Dense)          (None, 256)               64768     
                                                                 
 dropout_4318 (Dropout)      (None, 256)               0         
                                                                 
 dense_6097 (Dense)          (None, 16)                4112      
                                                                 
 dropout_4319 (Dropout)      (None, 16)                0         
                                                                 
 dense_6098 (Dense)          (None, 32)                544       
                                                                 
 dropout_4320 (Dropout)      (None, 32)                0         
                                                                 
 dense_6099 (Dense)          (None, 1)             

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=-0.543,RMSE=-51.625
Finished: 2022-09-16 19:56:54.732906


Could not learn, nor generalize

## PolyFeatures(2) + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=2,
                                verbose=0, callbacks=[es],  
                                ),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa4863d6610>], model=<function create_model at 0x7fa4888839e0>, model__degree=2, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa48ae51550>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48659e310>], verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fa5acad2c20>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

# CONTINUE HERE

In [ ]:
# ~20min
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__model__degree": [2],
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": all_permutations(layer_sizes, 3),
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits


Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 119},
'trf_reg__regressor__validation_split': 0.1, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'trf_reg__regressor__model__learning_rate': 0.001, 
'trf_reg__regressor__model__layer_nodes': [128, 32], 
'trf_reg__regressor__model__dropout': 0.1, 
'trf_reg__regressor__model__degree': 2, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 50, 
'trf_reg__regressor__batch_size': 32, 
'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=2,include_bias=False)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback], 
                                model__print_summary=True,   
                                validation_split=0.1, 
                                model__optim=RMSprop,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[128,32],
                                model__dropout=0.1,
                                model__degree=2,
                                model__activation="relu",
                                epochs=50, 
                                batch_size=32),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':119})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>, <keras.callbacks.LambdaCallback object at 0x7fa520152cd0>], epochs=50, model=<function create_model...cs=[<keras.metrics.RootMeanSquaredError object at 0x7fa48cd1e690>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48cd1edd0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.1, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fa5acad2c20>,
                                                                            kw_args={'a_max': 119,
                                                               

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_507"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1521 (Dense)          (None, 128)               32384     
                                                                 
 dropout_1014 (Dropout)      (None, 128)               0         
                                                                 
 dense_1522 (Dense)          (None, 32)                4128      
                                                                 
 dropout_1015 (Dropout)      (None, 32)                0         
                                                                 
 dense_1523 (Dense)          (None, 1)                 33        
                                                                 
Total params: 36,545
Trainable params: 36,545
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=1413.037, rmse=37.590, r2=0.096; v_lo

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.840,RMSE=-15.687
Finished: 2022-09-16 17:00:08.047966


Best result so far

## PolyFeatures(3) + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=3,
                                verbose=0, callbacks=[es],  
                                ))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>], model=<function create_model at 0x7fa530f75e60>, model__degree=3, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa48c38d290>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48c38de50>], verbose=0))])

In [ ]:
# ~25min
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__degree": [3],
        "mlp__model__optim":[Adam,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": all_permutations(layer_sizes, 3),
        "mlp__model__activation": ["relu","tanh", "sigmoid"]    
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'scaler': MinMaxScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 'mlp__model__learning_rate': 0.

Best Model:
```
{'scaler': MinMaxScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'mlp__model__learning_rate': 0.0001, 
'mlp__model__layer_nodes': [256, 64], 
'mlp__model__dropout': 0.1, 
'mlp__model__degree': 3, 
'mlp__model__activation': 'relu', 
'mlp__epochs': 50, 
'mlp__batch_size': 32}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],
                                model__print_summary=True,   
                                validation_split=0.2, 
                                model__optim=RMSprop,
                                model__learning_rate=0.0001, 
                                model__layer_nodes=[256,64],
                                model__dropout=0.1,
                                model__degree=3,
                                model__activation="relu",
                                epochs=50, 
                                batch_size=32))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>, <keras.callbacks.LambdaCallback object at 0x7fa520152cd0>], epochs=50, model=<function create_model at 0x7fa530f75e60>, model__activation='relu', model__de...el__dropout=0.1, model__layer_nodes=[256, 64], model__learning_rate=0.0001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa49d378810>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa48e3923d0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.2, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_761"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2283 (Dense)          (None, 256)               518144    
                                                                 
 dropout_1522 (Dropout)      (None, 256)               0         
                                                                 
 dense_2284 (Dense)          (None, 64)                16448     
                                                                 
 dropout_1523 (Dropout)      (None, 64)                0         
                                                                 
 dense_2285 (Dense)          (None, 1)                 65        
                                                                 
Total params: 534,657
Trainable params: 534,657
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=4928.556, rmse=70.204, r2=-0.134; v

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.551,RMSE=-27.830
Finished: 2022-09-16 17:31:09.294675


Similar architecture to the Linear 1-layer and Linear 1-poly 1-layer. Better than 2-degree (like in MLP-1) 

## PolyFeatures(3) + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                model__degree=3,
                                verbose=0, callbacks=[es],  
                                ),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fa530f0c610>], model=<function create_model at 0x7fa530f75e60>, model__degree=3, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fa48c9bd6d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fa49c272090>], verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fa5acad2c20>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~10min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__model__degree": [3],
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": all_permutations(layer_sizes, 3),
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits


Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 
'trf_reg__regressor__validation_split': 0.5, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'trf_reg__regressor__model__learning_rate': 0.01, 
'trf_reg__regressor__model__layer_nodes': [16], 
'trf_reg__regressor__model__dropout': 0.2, 
'trf_reg__regressor__model__degree': 3, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 50, 
'trf_reg__regressor__batch_size': 32, 
'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3,include_bias=False)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback], 
                                model__print_summary=True,   
                                validation_split=0.5, 
                                model__optim=Adam,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[16],
                                model__dropout=0.2,
                                model__degree=3,
                                model__activation="relu",
                                epochs=50, 
                                batch_size=32),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':135})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f1959d5b8d0>, <keras.callbacks.LambdaCallback object at 0x7f195016cf10>], epochs=50, model=<function cre..._metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f18b1862b50>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f18b0092c50>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__print_summary=True, validation_split=0.5, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f19d590bc20>,
                                                                            kw_args={'a_max': 135,
                                                              

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_1015"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2030 (Dense)          (None, 16)                32384     
                                                                 
 dropout_1015 (Dropout)      (None, 16)                0         
                                                                 
 dense_2031 (Dense)          (None, 1)                 17        
                                                                 
Total params: 32,401
Trainable params: 32,401
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=1975.571, rmse=44.447, r2=0.027; v_loss=1402.159, v_rmse=37.445, v_r2=0.314; 
E 1	: loss=1257.777, rmse=35.465, r2=0.381; v_loss=1110.082, v_rmse=33.318, v_r2=0.457; 
E 2	: loss=1061.762, rmse=32.585, r2=0.477; v_loss=951.694, v_rmse=30.850, v_r2=0.534; 
E 3	: loss=926.742, rmse=30.442, r2=0.544; v_

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.782,RMSE=-19.248
Finished: 2022-09-16 12:36:55.847390


Didn't improve by using more features, actually worsened a bit

# Smoothing

In [ ]:
def exponential_smoothing(df, sensors, n_samples, alpha=0.4, unit_col='unit_number'):
    df = df.copy()
    # first, calculate the exponential weighted mean of desired sensors
    df[sensors] = df.groupby(unit_col)[sensors].apply(lambda x: x.ewm(alpha=alpha).mean())
    
    # second, drop first n_samples of each unit_nr to reduce filter delay
    def create_mask(data, samples):
        result = np.ones_like(data)
        result[0:samples] = 0
        return result
    
    mask = df.groupby(unit_col)[unit_col].transform(create_mask, samples=n_samples).astype(bool)
    df = df[mask]
    
    return df

In [ ]:
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735,4
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594,3
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333,2
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640,1


In [ ]:
ss_train = StandardScaler().fit_transform(train)
ss_train = pd.DataFrame(ss_train, columns=train.columns)
ss_train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,RUL
0,-1.728084,-1.565170,-0.315980,-1.372953,0.0,0.0,-1.721725,-0.134255,-0.925936,-1.776357e-15,...,-1.058890,-0.269071,-0.603816,-1.387779e-17,-0.781710,0.0,0.0,1.348493,1.194427,1.207796
1,-1.728084,-1.550652,0.872722,-1.031720,0.0,0.0,-1.061780,0.211528,-0.643726,-1.776357e-15,...,-0.363646,-0.642845,-0.275852,-1.387779e-17,-0.781710,0.0,0.0,1.016528,1.236922,1.193277
2,-1.728084,-1.536134,-1.961874,1.015677,0.0,0.0,-0.661813,-0.413166,-0.525953,-1.776357e-15,...,-0.919841,-0.551629,-0.649144,-1.387779e-17,-2.073094,0.0,0.0,0.739891,0.503423,1.178759
3,-1.728084,-1.521616,0.324090,-0.008022,0.0,0.0,-0.661813,-1.261314,-0.784831,-1.776357e-15,...,-0.224597,-0.520176,-1.971665,-1.387779e-17,-0.781710,0.0,0.0,0.352598,0.777792,1.164241
4,-1.728084,-1.507098,-0.864611,-0.690488,0.0,0.0,-0.621816,-1.251528,-0.301518,-1.776357e-15,...,-0.780793,-0.521748,-0.339845,-1.387779e-17,-0.136018,0.0,0.0,0.463253,1.059552,1.149723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,1.659204,1.265868,-0.178822,-1.031720,0.0,0.0,1.618000,1.216258,2.188375,-1.776357e-15,...,2.278282,-0.322542,1.425294,-1.387779e-17,2.446751,0.0,0.0,-1.805173,-2.921113,-1.507098
20627,1.659204,1.280386,-0.727453,-1.714186,0.0,0.0,1.717992,2.279706,2.738351,-1.776357e-15,...,1.722087,-0.380207,1.913240,-1.387779e-17,1.155367,0.0,0.0,-2.856395,-1.203764,-1.521616
20628,1.659204,1.294904,0.186933,-0.008022,0.0,0.0,1.478011,1.946971,2.138377,-1.776357e-15,...,2.000184,-0.141684,3.265092,-1.387779e-17,3.092444,0.0,0.0,-2.081810,-3.292481,-1.536134
20629,1.659204,1.309423,-0.498857,1.015677,0.0,0.0,1.098043,2.403666,1.955051,-1.776357e-15,...,1.861136,-0.233948,2.579834,-1.387779e-17,1.155367,0.0,0.0,-2.911722,-2.085072,-1.550652


In [ ]:
(ss_train['s_2']-e_ss_train['s_2']).mean()

0.09587839238736667

In [ ]:
e_ss_train = exponential_smoothing(ss_train, ["s_0","s_2"], 0, 0.1)
e_ss_train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,RUL
0,-1.728084,-1.565170,-0.315980,-1.372953,0.0,0.0,-1.721725,-0.134255,-0.925936,-1.776357e-15,...,-1.058890,-0.269071,-0.603816,-1.387779e-17,-0.781710,0.0,0.0,1.348493,1.194427,1.207796
1,-1.728084,-1.550652,0.872722,-1.031720,0.0,0.0,-1.061780,0.047736,-0.643726,-1.776357e-15,...,-0.363646,-0.642845,-0.275852,-1.387779e-17,-0.781710,0.0,0.0,1.016528,1.236922,1.193277
2,-1.728084,-1.536134,-1.961874,1.015677,0.0,0.0,-0.661813,-0.122338,-0.525953,-1.776357e-15,...,-0.919841,-0.551629,-0.649144,-1.387779e-17,-2.073094,0.0,0.0,0.739891,0.503423,1.178759
3,-1.728084,-1.521616,0.324090,-0.008022,0.0,0.0,-0.661813,-0.453532,-0.784831,-1.776357e-15,...,-0.224597,-0.520176,-1.971665,-1.387779e-17,-0.781710,0.0,0.0,0.352598,0.777792,1.164241
4,-1.728084,-1.507098,-0.864611,-0.690488,0.0,0.0,-0.621816,-0.648398,-0.301518,-1.776357e-15,...,-0.780793,-0.521748,-0.339845,-1.387779e-17,-0.136018,0.0,0.0,0.463253,1.059552,1.149723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,1.659204,1.265868,-0.178822,-1.031720,0.0,0.0,1.618000,1.361246,2.188375,-1.776357e-15,...,2.278282,-0.322542,1.425294,-1.387779e-17,2.446751,0.0,0.0,-1.805173,-2.921113,-1.507098
20627,1.659204,1.280386,-0.727453,-1.714186,0.0,0.0,1.717992,1.453092,2.738351,-1.776357e-15,...,1.722087,-0.380207,1.913240,-1.387779e-17,1.155367,0.0,0.0,-2.856395,-1.203764,-1.521616
20628,1.659204,1.294904,0.186933,-0.008022,0.0,0.0,1.478011,1.502480,2.138377,-1.776357e-15,...,2.000184,-0.141684,3.265092,-1.387779e-17,3.092444,0.0,0.0,-2.081810,-3.292481,-1.536134
20629,1.659204,1.309423,-0.498857,1.015677,0.0,0.0,1.098043,1.592599,1.955051,-1.776357e-15,...,1.861136,-0.233948,2.579834,-1.387779e-17,1.155367,0.0,0.0,-2.911722,-2.085072,-1.550652


# OFF 3

## PolyFeatures(2) + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'  ,   MinMaxScaler()),
    ('mlp'  ,   KerasRegressor(model=create_model_pol,verbose=0, callbacks=[es], 
                               validation_split=0.2, 
                               model__metrics=[RMSE(), R2()],
                               model__loss='mse'))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()), ('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>], model=<function create_model at 0x7f36b04c3320>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f362c2ea850>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f36b0528250>], validation_split=0.2, verbose=0))])

In [ ]:
# ~6min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__optim":[Adam,SGD,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "mlp__model__activation": ["relu","tanh", "sigmoid"]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits


----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.001, 'mlp__model__layer_nodes': [128, 256, 512], 'mlp__model__dropout': 0.4, 'mlp__model__activation': 'sigmoid', 'mlp__epochs': 50, 'mlp__batch_size': 512}
Pipeline(steps=[('poly_ft', PolynomialFeatures()), ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=512, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36281d8550>], epochs=50, model=<function create_model at 0x7f36b04c3320>, model__activation='sigmoid', model__dropout

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.001, 
'mlp__model__layer_nodes': [128, 256, 512], 
'mlp__model__dropout': 0.4, 
'mlp__model__activation': 'sigmoid', 
'mlp__epochs': 50, 
'mlp__batch_size': 512}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model_pol, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.2, 
                                model__optim=SGD,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[128,256,512],
                                model__dropout=0.4,
                                model__activation='sigmoid',
                                epochs=50, 
                                batch_size=512))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()), ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=512, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>, <keras.callbacks.LambdaCallback object at 0x7f36c01c4850>], epochs=50, model=<function create_model at 0x7f36b04c3320>, model__activation='sigmoid', model__dropout=0.4, model__layer_nodes=[128, 256, 512], model__learning_rate=0.001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f361648bd50>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f361648b850>], model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, validation_split=0.2, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

E 0	: loss=4271.573, rmse=65.357, r2=0.017; v_loss=3707.898, v_rmse=60.893, v_r2=0.401; 
E 1	: loss=1938.243, rmse=44.025, r2=0.554; v_loss=2839.112, v_rmse=53.283, v_r2=0.541; 
E 2	: loss=1759.316, rmse=41.944, r2=0.595; v_loss=2711.072, v_rmse=52.068, v_r2=0.562; 
E 3	: loss=1734.720, rmse=41.650, r2=0.601; v_loss=2709.544, v_rmse=52.053, v_r2=0.562; 
E 4	: loss=1716.355, rmse=41.429, r2=0.605; v_loss=2621.782, v_rmse=51.203, v_r2=0.576; 
E 5	: loss=1714.445, rmse=41.406, r2=0.606; v_loss=2598.654, v_rmse=50.977, v_r2=0.580; 
E 6	: loss=1699.979, rmse=41.231, r2=0.609; v_loss=2570.351, v_rmse=50.699, v_r2=0.585; 
E 7	: loss=1699.774, rmse=41.228, r2=0.609; v_loss=2664.598, v_rmse=51.620, v_r2=0.569; 
E 8	: loss=1669.670, rmse=40.862, r2=0.616; v_loss=2607.967, v_rmse=51.068, v_r2=0.579; 
E 9	: loss=1662.341, rmse=40.772, r2=0.618; v_loss=2865.036, v_rmse=53.526, v_r2=0.537; 
E 10	: loss=1645.317, rmse=40.563, r2=0.622; v_loss=2657.781, v_rmse=51.554, v_r2=0.570; 
E 11	: loss=1641.543

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.469,RMSE=-30.277
Finished: 2022-09-12 10:50:06.977277


## PolyFeatures(3) + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'  ,   MinMaxScaler()),
    ('mlp'  ,   KerasRegressor(model=create_model_pol3,verbose=0, callbacks=[es], 
                               validation_split=0.2, 
                               model__metrics=[RMSE(), R2()],
                               model__loss='mse'))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>], model=<function create_model_pol3 at 0x7f361b2a1dd0>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f361b402f10>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f361b402090>], validation_split=0.2, verbose=0))])

In [ ]:
# ~6min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__optim":[Adam,SGD,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "mlp__model__activation": ["relu","tanh", "sigmoid"]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.001, 'mlp__model__layer_nodes': [128, 256, 512], 'mlp__model__dropout': 0.4, 'mlp__model__activation': 'sigmoid', 'mlp__epochs': 50, 'mlp__batch_size': 512}
Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
     

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>,
'mlp__model__learning_rate': 0.001, 
'mlp__model__layer_nodes': [128, 256, 512], 
'mlp__model__dropout': 0.4, 
'mlp__model__activation': 'sigmoid', 
'mlp__epochs': 50, 
'mlp__batch_size': 512}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model_pol3, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.2, 
                                model__optim=SGD,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[128,256,512],
                                model__dropout=0.4,
                                model__activation='sigmoid',
                                epochs=50, 
                                batch_size=512))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=512, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>, <keras.callbacks.LambdaCallback object at 0x7f36c01c4850>], epochs=50, model=<function create_model_pol3 at 0x7f361b2a1dd0>, model__activation='sigmoid', model__dropout=0.4, model__layer_nodes=[128, 256, 512], model__learning_rate=0.001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f3628307ad0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f3628307a50>], model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, validation_split=0.2, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

E 0	: loss=3859.454, rmse=62.125, r2=0.112; v_loss=3244.542, v_rmse=56.961, v_r2=0.476; 
E 1	: loss=1869.120, rmse=43.233, r2=0.570; v_loss=3304.891, v_rmse=57.488, v_r2=0.466; 
E 2	: loss=1816.944, rmse=42.626, r2=0.582; v_loss=2708.521, v_rmse=52.043, v_r2=0.562; 
E 3	: loss=1755.046, rmse=41.893, r2=0.596; v_loss=2778.461, v_rmse=52.711, v_r2=0.551; 
E 4	: loss=1729.353, rmse=41.585, r2=0.602; v_loss=2693.361, v_rmse=51.898, v_r2=0.565; 
E 5	: loss=1723.992, rmse=41.521, r2=0.603; v_loss=2670.318, v_rmse=51.675, v_r2=0.568; 
E 6	: loss=1698.269, rmse=41.210, r2=0.609; v_loss=2668.869, v_rmse=51.661, v_r2=0.569; 
E 7	: loss=1697.971, rmse=41.206, r2=0.609; v_loss=2667.741, v_rmse=51.650, v_r2=0.569; 
E 8	: loss=1675.807, rmse=40.937, r2=0.615; v_loss=2689.505, v_rmse=51.860, v_r2=0.565; 
E 9	: loss=1668.688, rmse=40.850, r2=0.616; v_loss=3041.413, v_rmse=55.149, v_r2=0.508; 
E 10	: loss=1650.078, rmse=40.621, r2=0.620; v_loss=2714.094, v_rmse=52.097, v_r2=0.561; 
E 11	: loss=1645.906

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.401,RMSE=-32.164
Finished: 2022-09-12 11:33:45.020869


Mesmo modelo do Poly2

## PolyFeatures(2) + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol,verbose=0, callbacks=[es], 
                                validation_split=0.2, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()), ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>], model=<function create_model_pol at 0x7f36162f0cb0>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f36286c33d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f36286c3b50>], validation_split=0.2, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f373c87e4d0>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~10min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 'trf_reg__regressor__validation_split': 0.5, 'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 'trf_reg__regressor__model__learning_rate': 0.01, 'trf_reg__regressor__model__layer_nodes': [64, 128, 256], 'trf_reg__regressor__model__dropout': 0.5, 'trf_reg__regressor__model__activation': 'tanh', 

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 
'trf_reg__regressor__validation_split': 0.5, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'trf_reg__regressor__model__learning_rate': 0.01, 
'trf_reg__regressor__model__layer_nodes': [64, 128, 256], 
'trf_reg__regressor__model__dropout': 0.5, 
'trf_reg__regressor__model__activation': 'tanh', 
'trf_reg__regressor__epochs': 30, 
'trf_reg__regressor__batch_size': 256, 
'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.5, 
                                model__optim=Adam,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[64, 128, 256],
                                model__dropout=0.5,
                                model__activation="tanh",
                                epochs=30, 
                                batch_size=256),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':135})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()), ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=256, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>, <keras.callbacks.LambdaCallback object at 0x7f36c01c4850>], epochs=30, model=<function create_model_pol at 0x7f36162f0...1, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f36165ef7d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f36165ef690>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, validation_split=0.5, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f373c87e4d0>,
                                                                            kw_args={'a_max': 135,
                                                                            

In [ ]:
# model.fit(X_train, y_train)
# reclipped_y =  model['trf_reg'].transformer.transform(y_train)
# eval.show_result(reclipped_y, model.predict(X_train))
# eval.show_result_cv(reclipped_y, X_train, model)

model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())
# eval.show_result_cv(y_train, X_train, model)

E 0	: loss=5314.098, rmse=72.898, r2=-1.617; v_loss=1288.184, v_rmse=35.891, v_r2=0.370; 
E 1	: loss=812.940, rmse=28.512, r2=0.600; v_loss=665.227, v_rmse=25.792, v_r2=0.675; 
E 2	: loss=576.975, rmse=24.020, r2=0.716; v_loss=567.214, v_rmse=23.816, v_r2=0.722; 
E 3	: loss=526.989, rmse=22.956, r2=0.741; v_loss=540.345, v_rmse=23.245, v_r2=0.736; 
E 4	: loss=509.114, rmse=22.564, r2=0.749; v_loss=530.718, v_rmse=23.037, v_r2=0.740; 
E 5	: loss=509.515, rmse=22.572, r2=0.749; v_loss=527.402, v_rmse=22.965, v_r2=0.742; 
E 6	: loss=506.129, rmse=22.497, r2=0.751; v_loss=529.495, v_rmse=23.011, v_r2=0.741; 
E 7	: loss=498.015, rmse=22.316, r2=0.755; v_loss=560.912, v_rmse=23.684, v_r2=0.726; 
E 8	: loss=495.849, rmse=22.268, r2=0.756; v_loss=550.318, v_rmse=23.459, v_r2=0.731; 
E 9	: loss=479.884, rmse=21.906, r2=0.764; v_loss=544.229, v_rmse=23.329, v_r2=0.734; 
E 10	: loss=492.787, rmse=22.199, r2=0.757; v_loss=545.064, v_rmse=23.347, v_r2=0.733; 
E 11	: loss=492.276, rmse=22.187, r2=0.

In [ ]:
# reclipped_y =  model['trf_reg'].transformer.transform(y_test)
# eval.show_result(reclipped_y, model.predict(X_test))

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.753,RMSE=-20.475
Finished: 2022-09-12 11:06:20.553177


## PolyFeatures(3) + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol3,verbose=0, callbacks=[es], 
                                validation_split=0.2, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>], model=<function create_model_pol3 at 0x7f361d4e1ef0>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f361d5f16d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f36167e2750>], validation_split=0.2, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f373c87e4d0>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~10min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                             factor=2, scorer='r2', ignore_warnings=True, cv=3)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 133}, 'trf_reg__regressor__validation_split': 0.1, 'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 'trf_reg__regressor__model__learning_rate': 0.001, 'trf_reg__regressor__model__layer_nodes': [64, 128, 256], 'trf_reg__regressor__model__dropout': 0.1, 'trf_reg__regressor__model__activation': 'relu',

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 133}, 
'trf_reg__regressor__validation_split': 0.1, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'trf_reg__regressor__model__learning_rate': 0.001, 
'trf_reg__regressor__model__layer_nodes': [64, 128, 256], 
'trf_reg__regressor__model__dropout': 0.1, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 30,
 'trf_reg__regressor__batch_size': 64, 
 'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol3, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.1, 
                                model__optim=Adam,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[64, 128, 256],
                                model__dropout=0.1,
                                model__activation="relu",
                                epochs=30, 
                                batch_size=64),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':133})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=64, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>, <keras.callbacks.LambdaCallback object at 0x7f36c01c4850>], epochs=30, model=<function create_model_pol3 at 0x7...1, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f3628705c90>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f3642112b50>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, validation_split=0.1, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f373c87e4d0>,
                                                                            kw_args={'a_max': 133,
                                                            

In [ ]:
# model.fit(X_train, y_train)
# reclipped_y =  model['trf_reg'].transformer.transform(y_train)
# eval.show_result(reclipped_y, model.predict(X_train))
# eval.show_result_cv(reclipped_y, X_train, model)

model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())
# eval.show_result_cv(y_train, X_train, model)

E 0	: loss=1465.280, rmse=38.279, r2=0.260; v_loss=877.040, v_rmse=29.615, v_r2=0.550; 
E 1	: loss=758.275, rmse=27.537, r2=0.617; v_loss=639.657, v_rmse=25.291, v_r2=0.672; 
E 2	: loss=628.628, rmse=25.072, r2=0.682; v_loss=544.522, v_rmse=23.335, v_r2=0.721; 
E 3	: loss=556.142, rmse=23.583, r2=0.719; v_loss=592.383, v_rmse=24.339, v_r2=0.696; 
E 4	: loss=532.007, rmse=23.065, r2=0.731; v_loss=452.094, v_rmse=21.263, v_r2=0.768; 
E 5	: loss=501.482, rmse=22.394, r2=0.747; v_loss=453.017, v_rmse=21.284, v_r2=0.767; 
E 6	: loss=502.090, rmse=22.407, r2=0.746; v_loss=424.875, v_rmse=20.612, v_r2=0.782; 
E 7	: loss=489.451, rmse=22.124, r2=0.753; v_loss=482.244, v_rmse=21.960, v_r2=0.752; 
E 8	: loss=482.443, rmse=21.965, r2=0.756; v_loss=482.506, v_rmse=21.966, v_r2=0.752; 
E 9	: loss=481.390, rmse=21.941, r2=0.757; v_loss=438.913, v_rmse=20.950, v_r2=0.775; 
E 10	: loss=481.096, rmse=21.934, r2=0.757; v_loss=506.945, v_rmse=22.515, v_r2=0.740; 
E 11	: loss=480.237, rmse=21.914, r2=0.75

In [ ]:
# reclipped_y =  model['trf_reg'].transformer.transform(y_test)
# eval.show_result(reclipped_y, model.predict(X_test))

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.811,RMSE=-17.814
Finished: 2022-09-12 11:21:51.443498


# OFF

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol3_iter,verbose=0, callbacks=[es], 
                                validation_split=0.2, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f8ad2298910>], model=<function create_model_pol3_iter at 0x7f8ad2405e60>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f8ad2328d90>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f8b32153150>], validation_split=0.2, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f8b4de4ec20>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [ ]:
list(powerset([16,32,64,128,256,512]))

[(),
 (16,),
 (32,),
 (64,),
 (128,),
 (256,),
 (512,),
 (16, 32),
 (16, 64),
 (16, 128),
 (16, 256),
 (16, 512),
 (32, 64),
 (32, 128),
 (32, 256),
 (32, 512),
 (64, 128),
 (64, 256),
 (64, 512),
 (128, 256),
 (128, 512),
 (256, 512),
 (16, 32, 64),
 (16, 32, 128),
 (16, 32, 256),
 (16, 32, 512),
 (16, 64, 128),
 (16, 64, 256),
 (16, 64, 512),
 (16, 128, 256),
 (16, 128, 512),
 (16, 256, 512),
 (32, 64, 128),
 (32, 64, 256),
 (32, 64, 512),
 (32, 128, 256),
 (32, 128, 512),
 (32, 256, 512),
 (64, 128, 256),
 (64, 128, 512),
 (64, 256, 512),
 (128, 256, 512),
 (16, 32, 64, 128),
 (16, 32, 64, 256),
 (16, 32, 64, 512),
 (16, 32, 128, 256),
 (16, 32, 128, 512),
 (16, 32, 256, 512),
 (16, 64, 128, 256),
 (16, 64, 128, 512),
 (16, 64, 256, 512),
 (16, 128, 256, 512),
 (32, 64, 128, 256),
 (32, 64, 128, 512),
 (32, 64, 256, 512),
 (32, 128, 256, 512),
 (64, 128, 256, 512),
 (16, 32, 64, 128, 256),
 (16, 32, 64, 128, 512),
 (16, 32, 64, 256, 512),
 (16, 32, 128, 256, 512),
 (16, 64, 128, 256

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
def run_RS(base_model, X_train, y_train, param_distributions, 
                  print_best=True, ignore_warnings=False, 
                  scorer=reclipper_scorer, cv=5):
        search = RandomizedSearchCV(base_model, param_distributions, 
                                    scoring=scorer, random_state=42, 
                                    verbose=1, cv=cv)
        search.fit(X_train, y_train);

        if(print_best): print("Best params: ", search.best_params_)
        return search.best_estimator_, search

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": [
            [16],[32],[64],[128],[256],[512],
            [16,32],[32,64],[64,128],[128,256],[256,512],
            [16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]
        ],
        # "trf_reg__regressor__model__layer_nodes": [
        #     #[16],[32],[64],[128],[256],[512],
        #     [16,32],[32,64],[64,128],[128,256],[256,512]
        #     #[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]
        # ],
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model,ss = run_RS(model, X_train, y_train, param_distributions, 
                          scorer='r2', ignore_warnings=True, cv=2)
    print(model)
    print("Finished:", datetime.datetime.now())

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 147}, 'trf_reg__regressor__validation_split': 0.1, 'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 'trf_reg__regressor__model__learning_rate': 0.001, 'trf_reg__regressor__model__layer_nodes': [256, 512], 'trf_reg__regressor__model__dropout': 0.1, 'trf_reg__regressor__model__activation': 'tanh', 'trf_reg__regressor__epochs': 5, 'trf_reg__regressor__batch_size': 32, 'scaler': StandardScaler()}
Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f8a32566590>], epochs=5, model=<function create_model_pol3_iter at 0x7f8ad2405e60>, model__activation='

In [ ]:
ss.cv_results_

{'mean_fit_time': array([ 8.09345293,  3.75099039,  6.387573  , 19.48472381,  8.97599912,
        34.48569763,  1.81102073, 10.1499145 ,  9.87086678,  6.23128748]),
 'std_fit_time': array([1.07878256e+00, 6.74605370e-03, 9.24661160e-02, 2.19633102e+00,
        2.48655677e+00, 7.67754614e+00, 3.31569314e-01, 1.19185877e+00,
        1.73339558e+00, 8.26859474e-03]),
 'mean_score_time': array([2.26794171, 0.59059727, 0.60934365, 0.67661905, 0.99048424,
        0.88606107, 0.43230832, 1.66806364, 0.67616022, 0.60011804]),
 'std_score_time': array([1.81133842e+00, 7.66042471e-02, 1.76775455e-03, 1.94716454e-03,
        2.31766701e-03, 1.14415288e-01, 4.81307507e-03, 9.92515802e-01,
        1.38294697e-03, 7.71039724e-02]),
 'param_trf_reg__transformer__kw_args': masked_array(data=[{'a_min': 0, 'a_max': 136}, {'a_min': 0, 'a_max': 116},
                    {'a_min': 0, 'a_max': 128}, {'a_min': 0, 'a_max': 97},
                    {'a_min': 0, 'a_max': 147}, {'a_min': 0, 'a_max': 115},
      

In [ ]:
sorted(list(zip(ss.cv_results_["mean_test_score"],ss.cv_results_["params"])), reverse=True)

[(0.5372751116225745,
  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 147},
   'trf_reg__regressor__validation_split': 0.1,
   'trf_reg__regressor__model__optim': keras.optimizer_v2.rmsprop.RMSprop,
   'trf_reg__regressor__model__learning_rate': 0.001,
   'trf_reg__regressor__model__layer_nodes': [256, 512],
   'trf_reg__regressor__model__dropout': 0.1,
   'trf_reg__regressor__model__activation': 'tanh',
   'trf_reg__regressor__epochs': 5,
   'trf_reg__regressor__batch_size': 32,
   'scaler': StandardScaler()}),
 (0.5046107971957956,
  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 136},
   'trf_reg__regressor__validation_split': 0.4,
   'trf_reg__regressor__model__optim': keras.optimizer_v2.adam.Adam,
   'trf_reg__regressor__model__learning_rate': 0.01,
   'trf_reg__regressor__model__layer_nodes': [256, 512],
   'trf_reg__regressor__model__dropout': 0.2,
   'trf_reg__regressor__model__activation': 'tanh',
   'trf_reg__regressor__epochs': 30,
   'trf_reg__regressor__

In [ ]:
ss.cv_results_["params"]

[{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 136},
  'trf_reg__regressor__validation_split': 0.4,
  'trf_reg__regressor__model__optim': keras.optimizer_v2.adam.Adam,
  'trf_reg__regressor__model__learning_rate': 0.01,
  'trf_reg__regressor__model__layer_nodes': [256, 512],
  'trf_reg__regressor__model__dropout': 0.2,
  'trf_reg__regressor__model__activation': 'tanh',
  'trf_reg__regressor__epochs': 30,
  'trf_reg__regressor__batch_size': 256,
  'scaler': StandardScaler()},
 {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 116},
  'trf_reg__regressor__validation_split': 0.1,
  'trf_reg__regressor__model__optim': keras.optimizer_v2.adam.Adam,
  'trf_reg__regressor__model__learning_rate': 0.01,
  'trf_reg__regressor__model__layer_nodes': [128, 256, 512],
  'trf_reg__regressor__model__dropout': 0.1,
  'trf_reg__regressor__model__activation': 'tanh',
  'trf_reg__regressor__epochs': 5,
  'trf_reg__regressor__batch_size': 128,
  'scaler': StandardScaler()},
 {'trf_reg__tran

In [ ]:
# ~5min
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        # "trf_reg__regressor__model__layer_nodes": [
        #     #[16],[32],[64],[128],[256],[512],
        #     [16,32],[32,64],[64,128],[128,256],[256,512]
        #     #[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]
        # ],
        "trf_reg__regressor__model__layer_nodes": [
            #[16],[32],[64],[128],[256],[512],
            [16,32],[32,64],[64,128],[128,256],[256,512]
            #[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]
        ],
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True, cv=2)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 133}, 'trf_reg__regressor__validation_split': 0.1, 'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 'trf_reg__regressor__model__learning_rate': 0.001, 'trf_reg__regressor__model__layer_nodes': [64, 128], 'trf_reg__regressor__model__dropout': 0.1, 'trf_reg__regressor__model__activation': 'relu', 'trf



```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 
'trf_reg__regressor__validation_split': 0.5, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'trf_reg__regressor__model__learning_rate': 0.01, 
'trf_reg__regressor__model__layer_nodes': [16], 
'trf_reg__regressor__model__dropout': 0.2, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 50, 
'trf_reg__regressor__batch_size': 32,
 'scaler': StandardScaler()}
```



In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol3_iter, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.5, 
                                model__optim=Adam,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[16],
                                model__dropout=0.2,
                                model__activation="relu",
                                model__print_summary=True,
                                epochs=50, 
                                batch_size=32),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':135})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7fdf147adc90>, <keras.callbacks.LambdaCallback object at 0x7fdf147a5d50>], epochs=10, model=<function create_model_pol3_iter..._metrics=[<keras.metrics.RootMeanSquaredError object at 0x7fdf14399450>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7fde97049590>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, model__print_summary=True, validation_split=0.1, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fe0443c3c20>,
                                                                            kw_args={'a_max': 133,
                                                               



```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 133}, 
'trf_reg__regressor__validation_split': 0.1, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'trf_reg__regressor__model__learning_rate': 0.001, 
'trf_reg__regressor__model__layer_nodes': [64, 128], 
'trf_reg__regressor__model__dropout': 0.1, 
'trf_reg__regressor__model__activation': 'relu',
 'trf_reg__regressor__epochs': 10, 
 'trf_reg__regressor__batch_size': 32,
  'scaler': StandardScaler()}
```



In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol3_iter, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.1, 
                                model__optim=Adam,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[64,128],
                                model__dropout=0.2,
                                model__activation="relu",
                                model__print_summary=True,
                                epochs=10, 
                                batch_size=32),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':133})))
])
model

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_427"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1178 (Dense)          (None, 64)                129600    
                                                                 
 dropout_752 (Dropout)       (None, 64)                0         
                                                                 
 dense_1179 (Dense)          (None, 128)               8320      
                                                                 
 dropout_753 (Dropout)       (None, 128)               0         
                                                                 
 dense_1180 (Dense)          (None, 1)                 129       
                                                                 
Total params: 138,049
Trainable params: 138,049
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=1524.700, rmse=39.047, r2=0.230; v_

# OFF 2

In [ ]:
import itertools

list(itertools.combinations([16,32,64], 2))

[(16, 32), (16, 64), (32, 64)]

In [ ]:
param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": all_permutations([16,32,64,128,256,512], 2),
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV


s = HalvingRandomSearchCV(model, param_distributions, factor=2,
                               min_resources=500, scoring='r2', 
                               random_state=42, verbose=1, cv=3)


s.fit(X_train, y_train);
print("Best params: ", s.best_params_)

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits


----------
iter: 1
n_candidates: 21
n_resources: 1000
Fitting 3 folds for each of 21 candidates, totalling 63 fits
----------
iter: 2
n_candidates: 11
n_resources: 2000
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 3
n_candidates: 6
n_resources: 4000
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 4
n_candidates: 3
n_resources: 8000
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 5
n_candidates: 2
n_resources: 16000
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 119}, 'trf_reg__regressor__validation_split': 0.1, 'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 'trf_reg__regressor__model__learning_rate': 0.001, 'trf_reg__regressor__model__layer_nodes': [128, 32], 'trf_reg__regressor__model__dropout': 0.1, 'trf_reg__regressor__model__activation': 'relu', 'trf_reg__regressor__epochs': 50,



```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 122}, 
'trf_reg__regressor__validation_split': 0.4, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>, 
'trf_reg__regressor__model__learning_rate': 0.001, 'trf_reg__regressor__model__layer_nodes': [16, 256], 
'trf_reg__regressor__model__dropout': 0.2, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 20, 
'trf_reg__regressor__batch_size': 64, 
'scaler': StandardScaler()}
```





```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 119}, 
'trf_reg__regressor__validation_split': 0.1, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.rmsprop.RMSprop'>,
 'trf_reg__regressor__model__learning_rate': 0.001, 
 'trf_reg__regressor__model__layer_nodes': [128, 32], 
 'trf_reg__regressor__model__dropout': 0.1, 
 'trf_reg__regressor__model__activation': 'relu', 
 'trf_reg__regressor__epochs': 50,
  'trf_reg__regressor__batch_size': 32,
   'scaler': StandardScaler()}
```



In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol3_iter, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.1, 
                                model__optim=RMSprop,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[128,32],
                                model__dropout=0.1,
                                model__activation="relu",
                                model__print_summary=True,
                                epochs=50, 
                                batch_size=32),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':119})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f8ad2298910>, <keras.callbacks.LambdaCallback object at 0x7f8ad22a8f10>], epochs=50, model=<function create_model_pol3_iter...cs=[<keras.metrics.RootMeanSquaredError object at 0x7f8a24130950>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f8a32411810>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__print_summary=True, validation_split=0.1, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f8b4de4ec20>,
                                                                            kw_args={'a_max': 119,
                                                               

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())

Model: "sequential_280"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_850 (Dense)           (None, 128)               259200    
                                                                 
 dropout_570 (Dropout)       (None, 128)               0         
                                                                 
 dense_851 (Dense)           (None, 32)                4128      
                                                                 
 dropout_571 (Dropout)       (None, 32)                0         
                                                                 
 dense_852 (Dense)           (None, 1)                 33        
                                                                 
Total params: 263,361
Trainable params: 263,361
Non-trainable params: 0
_________________________________________________________________
E 0	: loss=1021.892, rmse=31.967, r2=0.346; v_

In [ ]:
model.fit(X_train, y_train)

E 0	: loss=5207.414, rmse=72.162, r2=-0.345; v_loss=3045.556, v_rmse=55.187, v_r2=0.451; 
E 1	: loss=1527.117, rmse=39.078, r2=0.606; v_loss=2813.525, v_rmse=53.043, v_r2=0.493; 
E 2	: loss=1359.615, rmse=36.873, r2=0.649; v_loss=2666.109, v_rmse=51.634, v_r2=0.519; 
E 3	: loss=1315.130, rmse=36.265, r2=0.660; v_loss=2877.303, v_rmse=53.640, v_r2=0.481; 
E 4	: loss=1290.427, rmse=35.923, r2=0.667; v_loss=2632.022, v_rmse=51.303, v_r2=0.526; 
E 5	: loss=1294.368, rmse=35.977, r2=0.666; v_loss=2702.958, v_rmse=51.990, v_r2=0.513; 
E 6	: loss=1239.708, rmse=35.209, r2=0.680; v_loss=3236.657, v_rmse=56.892, v_r2=0.417; 
E 7	: loss=1272.573, rmse=35.673, r2=0.671; v_loss=3314.008, v_rmse=57.567, v_r2=0.403; 
E 8	: loss=1270.919, rmse=35.650, r2=0.672; v_loss=2804.315, v_rmse=52.956, v_r2=0.494; 
E 9	: loss=1257.236, rmse=35.458, r2=0.675; v_loss=2682.475, v_rmse=51.793, v_r2=0.516; 
E 10	: loss=1252.234, rmse=35.387, r2=0.677; v_loss=2580.146, v_rmse=50.795, v_r2=0.535; 
E 11	: loss=1235.93

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(batch_size=128, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f64ae3187d0>, <keras.callbacks.LambdaCallback object at 0x7f64aebf8b10>], epochs=20, model=<function create_model at 0x7f6542c000e0>, model__activation='relu', model__dropout=0.1, model__layer_nodes=[16, 32, 64], model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f64ae30f6d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f64ae317350>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, validation_split=0.5, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
# eval.show_result_cv(y_train, X_train, model)

E 0	: loss=4787.087, rmse=61.810, r2=0.195; v_loss=3012.669, v_rmse=54.888, v_r2=0.457; 
E 1	: loss=1573.484, rmse=39.667, r2=0.594; v_loss=2781.886, v_rmse=52.744, v_r2=0.499; 
E 2	: loss=1422.959, rmse=37.722, r2=0.632; v_loss=2648.300, v_rmse=51.462, v_r2=0.523; 
E 3	: loss=1322.943, rmse=36.372, r2=0.658; v_loss=2887.253, v_rmse=53.733, v_r2=0.480; 
E 4	: loss=1293.984, rmse=35.972, r2=0.666; v_loss=2671.713, v_rmse=51.689, v_r2=0.518; 
E 5	: loss=1264.718, rmse=35.563, r2=0.673; v_loss=2667.296, v_rmse=51.646, v_r2=0.519; 
E 6	: loss=1209.704, rmse=34.781, r2=0.688; v_loss=3272.676, v_rmse=57.207, v_r2=0.410; 
E 7	: loss=1235.243, rmse=35.146, r2=0.681; v_loss=2919.450, v_rmse=54.032, v_r2=0.474; 
E 8	: loss=1193.297, rmse=34.544, r2=0.692; v_loss=2588.395, v_rmse=50.876, v_r2=0.533; 
E 9	: loss=1183.923, rmse=34.408, r2=0.694; v_loss=2595.500, v_rmse=50.946, v_r2=0.532; 
E 10	: loss=1186.067, rmse=34.439, r2=0.694; v_loss=2638.092, v_rmse=51.362, v_r2=0.524; 
E 11	: loss=1173.344

In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.643,RMSE=-24.825


In [ ]:
dir(search)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_input_parameters',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_checked_cv_orig',
 '_estimator_type',
 '_format_results',
 '_generate_candidate_params',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_n_samples_orig',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 'aggressive_elimination',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',


In [ ]:
search.score

<bound method BaseSearchCV.score of HalvingRandomSearchCV(cv=3,
                      estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                                ('mlp',
                                                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fdf39694cd0>], model=<function create_model at 0x7fdf49de63b0>, validation_split=0.2, verbose=0))]),
                      min_resources=500,
                      param_distributions={'mlp__epochs': [1, 5, 10, 20, 30,
                                                           50],
                                           'mlp__model__activation': ['relu'...
                                           'mlp__model__optim': [<class 'keras.optimizer_v2.adam.Adam'>,
                                                                 <class 'keras.optimizer_v2.gradient_descent.SGD'>,
                                                                 <class 'keras.optimizer_v2.rmsprop.R

## Linear RUL

In [ ]:
model = Pipeline([
    ('tree_reg'  ,   DecisionTreeRegressor(random_state=42))
])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "tree_reg__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "tree_reg__splitter": ["best", "random"],
        "tree_reg__max_depth": [None,100,200,300],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": ["sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__splitter': 'best', 'tree_reg__min_samples_split': 2, 'tree_reg__min_samples_leaf': 20, 'tree_reg__min_impurity_decrease': 3.5, 'tree_reg__max_features': 'log2', 'tree_reg__max_depth': 200, 'tree_reg__criterion': 'friedman_mse', 'tree_reg__ccp_alpha': 0.675}
Pipeline(steps=[('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=0.675,
                          

Best Model:
```
('tree_reg',
DecisionTreeRegressor(ccp_alpha=0.675,
                    criterion='friedman_mse', max_depth=200,
                    max_features='log2',
                    min_impurity_decrease=3.5,
                    min_samples_leaf=20, random_state=42))
```

In [ ]:
model = Pipeline([
    ('tree_reg'  ,   DecisionTreeRegressor(ccp_alpha=0.675,
                                    criterion='friedman_mse', max_depth=200,
                                    max_features='log2',
                                    min_impurity_decrease=3.5,
                                    min_samples_leaf=20, random_state=42))
])
model

Pipeline(steps=[('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=0.675,
                                       criterion='friedman_mse', max_depth=200,
                                       max_features='log2',
                                       min_impurity_decrease=3.5,
                                       min_samples_leaf=20, random_state=42))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.645,RMSE=-41.037
(CV) R2=0.537,RMSE=-46.205


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.298,RMSE=-34.821


Worse than simple linear regression

## Non-linear RUL

In [ ]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "trf_reg__regressor__splitter": ["best", "random"],
        "trf_reg__regressor__max_depth": [None,100,200,300],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": ["sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 98}, 'trf_reg__regressor__splitter': 'best', 'trf_reg__regressor__min_samples_split': 2, 'trf_reg__regressor__min_samples_leaf': 10, 'trf_reg__regressor__min_impurity_decrease': 5.5, 'trf_reg__regressor__max_features': 'sqrt', 'trf_reg__regressor__max_depth': 200, 'trf_reg__regressor__criterion': 'friedman_mse', 'trf_reg__r

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 98}}

DecisionTreeRegressor(ccp_alpha=0.375,
                    criterion='friedman_mse',
                    max_depth=200,
                    max_features='sqrt',
                    min_impurity_decrease=5.5,
                    min_samples_leaf=10,
                    random_state=42),
```



In [ ]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(ccp_alpha=0.375,
                            criterion='friedman_mse',
                            max_depth=200,
                            max_features='sqrt',
                            min_impurity_decrease=5.5,
                            min_samples_leaf=10,
                            random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':98})))
    ])
model

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DecisionTreeRegressor(ccp_alpha=0.375,
                                                                            criterion='friedman_mse',
                                                                            max_depth=200,
                                                                            max_features='sqrt',
                                                                            min_impurity_decrease=5.5,
                                                                            min_samples_leaf=10,
                                                                            random_state=42),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fd35e8e7cb0>,
                                                                            kw_args={'a_max': 98,
                

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.815,RMSE=-13.536
(CV) R2=0.781,RMSE=-14.680


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.819,RMSE=-14.277


The best result we had so far in training and in testing

## PolyFeatures + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('tree_reg'  ,   DecisionTreeRegressor(random_state=42))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('tree_reg', DecisionTreeRegressor(random_state=42))])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "tree_reg__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "tree_reg__splitter": ["best", "random"],
        "tree_reg__max_depth": [None,100,200,300],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": [None, "sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__splitter': 'random', 'tree_reg__min_samples_split': 15, 'tree_reg__min_samples_leaf': 15, 'tree_reg__min_impurity_decrease': 6.9, 'tree_reg__max_features': 'sqrt', 'tree_reg__max_depth': 100, 'tree_reg__criterion': 'squared_error', 'tree_reg__ccp_alpha': 1.025, 'poly_ft__interaction_only': False, 'poly_ft__include_bias': True, 'poly_ft__degree': 3}
Pipeline(steps=[('

Best Model:
```
PolynomialFeatures(degree=3)
DecisionTreeRegressor(ccp_alpha=1.025, max_depth=100,
                    max_features='sqrt',
                    min_impurity_decrease=6.9,
                    min_samples_leaf=15,
                    min_samples_split=15, random_state=42,
                    splitter='random'))
```


In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('tree_reg'  ,   DecisionTreeRegressor(ccp_alpha=1.025, max_depth=100,
                                        max_features='sqrt',
                                        min_impurity_decrease=6.9,
                                        min_samples_leaf=15,
                                        min_samples_split=15, random_state=42,
                                        splitter='random'))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=1.025, max_depth=100,
                                       max_features='sqrt',
                                       min_impurity_decrease=6.9,
                                       min_samples_leaf=15,
                                       min_samples_split=15, random_state=42,
                                       splitter='random'))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.580,RMSE=-44.656
(CV) R2=0.550,RMSE=-45.641


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.311,RMSE=-34.484


No clear benefit from polynomial features in linear RUL, as with the other models. Actually, it worsened the results.

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "trf_reg__regressor__splitter": ["best", "random"],
        "trf_reg__regressor__max_depth": [None,100,200,300],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": [None, "sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 71}, 'trf_reg__regressor__splitter': 'random', 'trf_reg__regressor__min_samples_split': 20, 'trf_reg__regressor__min_samples_leaf': 2, 'trf_reg__regressor__min_impurity_decrease': 8.5, 'trf_reg__regressor__max_features': None, 'trf_reg__regressor__max_depth': 200, 'trf_reg__regressor__criterion': 'squared_error', 'trf_reg__

Best Model:
```
'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 71}
PolynomialFeatures(degree=3)
DecisionTreeRegressor(ccp_alpha=1.85,
                        max_depth=200,
                        min_impurity_decrease=8.5,
                        min_samples_leaf=2,
                        min_samples_split=20,
                        random_state=42,
                        splitter='random'),
```


In [ ]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(ccp_alpha=1.85,
                                            max_depth=200,
                                            min_impurity_decrease=8.5,
                                            min_samples_leaf=2,
                                            min_samples_split=20,
                                            random_state=42,
                                            splitter='random'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':71})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DecisionTreeRegressor(ccp_alpha=1.85,
                                                                            max_depth=200,
                                                                            min_impurity_decrease=8.5,
                                                                            min_samples_leaf=2,
                                                                            min_samples_split=20,
                                                                            random_state=42,
                                                                            splitter='random'),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fd35e8e7cb0>,
                                                               

In [ ]:
# ~3min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.772,RMSE=-9.983
(CV) R2=0.772,RMSE=-9.954


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.711,RMSE=-12.322


Using the polynominal features actually worsened the perfomance a little bit, when comparing with not using them and the non-linear RUL.